In [1]:
import pandas as pd
import os
import pickle
import glob
import warnings

In [5]:
ret_sample_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv'

df = pd.read_csv(ret_sample_path)

print(df.columns)

C:\Users\carte\AppData\Local\Temp\ipykernel_29620\2979320123.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ret_sample_path)


Index(['id', 'date', 'ret_eom', 'gvkey', 'iid', 'excntry', 'stock_ret', 'year', 'month', 'char_date',
       ...
       'betadown_252d', 'prc_highprc_252d', 'corr_1260d', 'betabab_1260d', 'rmax5_rvol_21d', 'age', 'qmj', 'qmj_prof', 'qmj_growth', 'qmj_safety'], dtype='object', length=159)


In [2]:
ret_sample_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv'
pd.set_option("display.max_rows", 1000)
df = pd.read_csv(ret_sample_path, usecols=['date'])
print(df.sample(100))

             date
1695191  20101130
5021799  20210831
1109133  20081128
3941632  20180531
2214206  20120831
677865   20070629
3413785  20160831
4202991  20190329
405709   20060721
4193811  20190228
1377171  20091029
3649416  20170630
6369671  20250530
83198    20050531
3856221  20180228
449196   20060929
2960688  20150227
5884210  20240131
4654061  20200731
4325633  20190731
6295073  20250331
3201820  20151230
148835   20050831
2303098  20121228
1256409  20090529
4453515  20191231
1898712  20110729
621200   20070430
4542759  20200331
4155784  20190130
447084   20060929
5997297  20240531
104683   20050630
5108882  20211130
3042563  20150630
1326575  20090831
1114031  20081230
6386094  20250630
2774267  20140731
3925624  20180430
1761546  20110228
1014558  20080829
452082   20060928
1121113  20081230
1415676  20091230
195749   20051031
5188294  20220228
5629329  20230428
2481787  20130731
975004   20080630
2273765  20121129
5051515  20210930
2377526  20130329
4369177  20190930
461199   2

In [8]:
import pandas as pd
import os

def process_csv_in_chunks(input_csv_path, output_dir, chunk_size=1000000):
    """
    Reads and processes a large CSV in chunks to avoid memory errors. It
    uses the 'gvkey' column to save separate CSV files for each unique ID.

    Args:
        input_csv_path (str): The path to the input CSV file.
        output_dir (str): The directory where the output CSVs will be saved.
        chunk_size (int): The number of rows to process in each chunk.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    # This set will keep track of which files we have already written the header for
    created_files = set()
    
    try:
        print(f"Reading data from {input_csv_path} in chunks of {chunk_size} rows...")
        
        # Use the 'chunksize' parameter to create an iterator
        chunk_iterator = pd.read_csv(input_csv_path, low_memory=False, chunksize=chunk_size)
        
        # Loop through each chunk (a smaller DataFrame) from the file
        for i, chunk in enumerate(chunk_iterator):
            print(f"--- Processing Chunk {i+1} ---")

            # --- ID Transformation using 'gvkey' column ---
            print("Cleaning 'gvkey' column and preparing for grouping...")
            # Drop rows where gvkey is missing
            chunk.dropna(subset=['gvkey'], inplace=True)
            # Convert gvkey to integer type to remove decimals (e.g., 1081.0 -> 1081)
            chunk['gvkey'] = chunk['gvkey'].astype(int)
            
            # --- Append rows to the correct files ---
            print("Grouping by 'gvkey' and saving to files...")
            # Group the current chunk by the integer 'gvkey'
            grouped_chunk = chunk.groupby('gvkey')
            
            # Loop through each unique ID found in this chunk
            for current_id, group_df in grouped_chunk:
                file_name = f"{current_id}.csv"
                output_path = os.path.join(output_dir, file_name)
                
                # If we haven't created this file yet, write with a header.
                # Otherwise, append without the header.
                if output_path not in created_files:
                    group_df.to_csv(output_path, index=False, mode='w', header=True)
                    created_files.add(output_path) # Mark this file as created
                else:
                    group_df.to_csv(output_path, index=False, mode='a', header=False)
            
            print(f"Chunk {i+1} processed successfully.")
            # Add a running counter after each chunk is processed
            print(f"Unique IDs found so far: {len(created_files)}")

        print("\nProcessing complete. All chunks have been processed and files saved.")
        # Add a final count at the end of the script
        print(f"Total unique IDs found and files created: {len(created_files)}")

    except FileNotFoundError:
        print(f"Error: The file '{input_csv_path}' was not found. Please check the path.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- How to use this script ---
if __name__ == "__main__":
    print("Starting the memory-efficient script to split data by company ID.")
    
    # 1. Set the path to your input CSV file.
    input_file_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv'
    
    # 2. Set the path for the output folder.
    output_folder_name = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\csv_by_id'
    
    # 3. Run the main function.
    process_csv_in_chunks(input_file_path, output_folder_name)



Starting the memory-efficient script to split data by company ID.
Created output directory: C:\_Files\Personal\Projects\FIAM\FIAM2025\data\csv_by_id
Reading data from C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv in chunks of 1000000 rows...
--- Processing Chunk 1 ---
Cleaning 'gvkey' column and preparing for grouping...
Grouping by 'gvkey' and saving to files...
Chunk 1 processed successfully.
Unique IDs found so far: 29651
--- Processing Chunk 2 ---
Cleaning 'gvkey' column and preparing for grouping...
Grouping by 'gvkey' and saving to files...
Chunk 2 processed successfully.
Unique IDs found so far: 33335
--- Processing Chunk 3 ---
Cleaning 'gvkey' column and preparing for grouping...
Grouping by 'gvkey' and saving to files...
Chunk 3 processed successfully.
Unique IDs found so far: 36931
--- Processing Chunk 4 ---
Cleaning 'gvkey' column and preparing for grouping...
Grouping by 'gvkey' and saving to files...
Chunk 4 processed successfully.
Unique IDs found so far: 

In [3]:
# rf and mgmnt are the risk factors and management discussion and analysis respectively. 
pd.set_option('display.max_columns', None)

# 2. Set display width: This is the key setting to prevent line wrapping.
#    We set it to a very large number to ensure pandas tries to fit everything on one line.
pd.set_option('display.width', 300) # You can increase this number if needed
pd.set_option('display.max_colwidth', 200)

# Optional: If you also want to see more rows in your output
pd.set_option('display.max_rows', 100) # This will show up to 100 rows

directory_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'

pickle_pattern = os.path.join(directory_path, '**', '*.pkl')
all_pickle_files = glob.glob(pickle_pattern, recursive=True)

if all_pickle_files:

    first_file_path = all_pickle_files[0]

    df = pd.read_pickle(first_file_path)

    print(df.head)
    print(df.columns)

<bound method NDFrame.head of             date      cik file_type                                                                                                                                                                                                       rf  \
177795  20050103    16099       10Q                                                                                                                                                                                                            
177791  20050103   779544       10K                                                                                                                                                                                                            
177794  20050103   831641       10K                                                                                                                                                                                                            
177790  20

In [1]:
# First, you need to install the 'transformers' library from Hugging Face.
# You can do this by opening your terminal or command prompt and running:
# pip install transformers torch

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def analyze_sentiment(text_list):
    """
    Analyzes the sentiment of a list of financial texts using a pre-trained FinBERT model.

    Args:
        text_list (list of str): A list of sentences to analyze.

    Returns:
        list of dict: A list of dictionaries, each containing the original text,
                      the predicted sentiment (Positive, Negative, Neutral),
                      and the model's confidence score.
    """
    # 1. Load the Pre-trained FinBERT Model and Tokenizer
    # =======================================================
    # We are using a version of FinBERT that has been specifically fine-tuned for
    # sentiment analysis on financial text.
    model_name = "ProsusAI/finbert"
    print(f"Loading model '{model_name}'... (This may take a moment on first run)")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    print("Model loaded successfully.")

    # 2. Tokenize the Input Text
    # ==========================
    # The tokenizer converts our human-readable sentences into a format of numbers (tokens)
    # that the model can understand. `padding=True` and `truncation=True` handle
    # sentences of different lengths.
    inputs = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")

    # 3. Get Model Predictions
    # ========================
    # We run the tokenized inputs through the model. `torch.no_grad()` is a performance
    # optimization that tells PyTorch we are not training the model, just using it.
    with torch.no_grad():
        outputs = model(**inputs)

    # 4. Interpret the Results
    # ========================
    # The model's output (logits) are raw scores. We apply a softmax function to
    # convert them into probabilities between 0 and 1.
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # The model was trained on three labels: positive, negative, neutral.
    labels = ["positive", "negative", "neutral"]
    
    results = []
    for i in range(len(text_list)):
        # Find the index of the highest probability score
        sentiment_idx = torch.argmax(predictions[i]).item()
        # Get the corresponding label
        sentiment = labels[sentiment_idx]
        # Get the confidence score for that prediction
        confidence = predictions[i][sentiment_idx].item()
        
        results.append({
            "text": text_list[i],
            "sentiment": sentiment.capitalize(),
            "confidence": f"{confidence:.2%}" # Format as percentage
        })
        
    return results

# --- How to use this script ---
if __name__ == "__main__":
    # Define some example financial sentences to analyze
    example_sentences = [
        "The company's earnings exceeded expectations, driven by strong sales growth.",
        "Regulatory concerns and rising costs have negatively impacted our outlook.",
        "The new factory is expected to begin operations in the third quarter.",
        "Despite a challenging market, the firm maintained its dividend payout.",
        "Profits have plummeted following the product recall."
    ]

    # Get the sentiment analysis results
    sentiment_results = analyze_sentiment(example_sentences)

    # Print the results in a clean format
    print("\n--- FinBERT Sentiment Analysis Results ---")
    for result in sentiment_results:
        print(f"  Sentence:   '{result['text']}'")
        print(f"  Sentiment:  {result['sentiment']} (Confidence: {result['confidence']})\n")


c:\_Files\Personal\Projects\FIAM\FIAM2025\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model 'ProsusAI/finbert'... (This may take a moment on first run)


c:\_Files\Personal\Projects\FIAM\FIAM2025\myenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\carte\.cache\huggingface\hub\models--ProsusAI--finbert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular

Model loaded successfully.

--- FinBERT Sentiment Analysis Results ---
  Sentence:   'The company's earnings exceeded expectations, driven by strong sales growth.'
  Sentiment:  Positive (Confidence: 95.96%)

  Sentence:   'Regulatory concerns and rising costs have negatively impacted our outlook.'
  Sentiment:  Negative (Confidence: 94.92%)

  Sentence:   'The new factory is expected to begin operations in the third quarter.'
  Sentiment:  Neutral (Confidence: 87.01%)

  Sentence:   'Despite a challenging market, the firm maintained its dividend payout.'
  Sentiment:  Positive (Confidence: 95.12%)

  Sentence:   'Profits have plummeted following the product recall.'
  Sentiment:  Negative (Confidence: 97.40%)



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [4]:
import os
import glob
import pickle
import pandas as pd # Import pandas to handle the DataFrame

def inspect_first_filing(data_directory, output_filename="filing_example.txt"):
    """
    Finds the first pickle file, searches for the first row with non-empty text
    sections, and saves that filing's text to a file for inspection.

    Args:
        data_directory (str): The root path where the text data is stored.
        output_filename (str): The name of the text file to create.
    """
    print(f"Searching for pickle files in '{data_directory}'...")
    
    # 1. Find the first pickle file automatically
    # ============================================
    pickle_pattern = os.path.join(data_directory, '**', '*.pkl')
    all_pickle_files = glob.glob(pickle_pattern, recursive=True)

    if not all_pickle_files:
        print("Error: No pickle files were found. Please check the directory path.")
        return

    first_file_path = all_pickle_files[0]
    print(f"Found files. Loading the first one: '{os.path.basename(first_file_path)}'")

    # 2. Load the data from the pickle file
    # =======================================
    try:
        with open(first_file_path, 'rb') as f:
            filings_df = pickle.load(f)
            
            if filings_df.empty:
                print("The pickle file contains an empty DataFrame.")
                return

            # --- NEW: Loop to find the first valid row ---
            # We'll search for the first row that has meaningful content in both key sections.
            valid_filing_series = None
            for index, row in filings_df.iterrows():
                # The correct column names are 'rf' (Risk Factors) and 'mgmt' (MD&A)
                risk_text = row.get('rf')
                mda_text = row.get('mgmt')

                # Check if both sections exist and have a reasonable length (e.g., > 100 characters)
                if pd.notna(risk_text) and len(str(risk_text).strip()) > 100 and \
                   pd.notna(mda_text) and len(str(mda_text).strip()) > 100:
                    valid_filing_series = row
                    print(f"Found a suitable filing for inspection at index {index}.")
                    break # Exit the loop once a good example is found
            
            if valid_filing_series is None:
                print("Error: Could not find a single filing with complete text sections in this file.")
                return

            # 3. Extract data from the VALID filing row
            # ==========================================================
            cik = valid_filing_series.get('cik', 'CIK not found')
            filing_date = valid_filing_series.get('date', 'Date not found')
            risk_factors_text = valid_filing_series.get('rf', 'Section not found.')
            mda_text = valid_filing_series.get('mgmt', 'Section not found.')

            print(f"Extracting data for CIK: {cik}, Filing Date: {filing_date}")


    except Exception as e:
        print(f"An error occurred while loading or processing the pickle file: {e}")
        return

    # 4. Write the extracted text to a plain text file
    # =================================================
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write(f"EXAMPLE FILING FOR INSPECTION\n")
            f.write(f"CIK: {cik} | DATE: {filing_date}\n")
            f.write("="*80 + "\n\n")

            f.write("--- RISK FACTORS ---\n\n")
            f.write(str(risk_factors_text)) # Use str() to handle potential non-string data
            f.write("\n\n")

            f.write("="*80 + "\n\n")

            f.write("--- MANAGEMENT'S DISCUSSION AND ANALYSIS (MD&A) ---\n\n")
            f.write(str(mda_text)) # Use str() to handle potential non-string data
        
        print(f"\nSuccess! The content has been written to '{output_filename}'")
        print("You can now open this file to see the raw text.")

    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


# --- How to use this script ---
if __name__ == "__main__":
    # !! IMPORTANT: Make sure this path points to your unzipped text data directory !!
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    
    inspect_first_filing(text_data_path)



Searching for pickle files in 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'...
Found files. Loading the first one: 'text_us_2005.pkl'


C:\Users\carte\AppData\Local\Temp\ipykernel_19912\3822597473.py:33: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  filings_df = pickle.load(f)


Found a suitable filing for inspection at index 179123.
Extracting data for CIK: 879526, Filing Date: 20050303

Success! The content has been written to 'filing_example.txt'
You can now open this file to see the raw text.


In [12]:
import os
import glob
import pickle
import pandas as pd
import re # Import the regular expression module

def is_title(line):
    """
    Heuristic to determine if a line of text is a title.
    A line is considered a title if it's all uppercase, doesn't end with a period,
    and has a reasonable length.
    """
    # Increased the word count limit to 35 to catch longer titles.
    return line.isupper() and len(line.split()) < 35 and not line.strip().endswith('.') and len(line) > 5

def parse_risk_factors(raw_text):
    """
    Parses the raw text of a 'Risk Factors' section to intelligently combine titles
    with their corresponding paragraphs. This version is more robust.

    Args:
        raw_text (str): The full text of the risk factors section.

    Returns:
        str: A formatted string with titles reliably merged with their paragraphs.
    """
    if not isinstance(raw_text, str):
        return "Invalid text input."

    # Split the entire text block into paragraphs based on one or more empty lines.
    raw_paragraphs = re.split(r'\n\s*\n', raw_text.strip())
    
    # Clean up whitespace, join any internally wrapped lines, and filter out page numbers.
    clean_paragraphs = []
    for para in raw_paragraphs:
        clean_para = ' '.join(para.split())
        # Add a check to filter out paragraphs that are just numbers (page numbers)
        if clean_para and not clean_para.isdigit():
            clean_paragraphs.append(clean_para)

    # --- New, More Robust Merging Logic ---
    parsed_chunks = []
    held_title = ""
    for para in clean_paragraphs:
        if is_title(para):
            # If we were already holding a title, it means we found two titles in a row.
            # The previous one must have been a standalone title or sub-header.
            if held_title:
                parsed_chunks.append(held_title)
            # Now, hold the new title, waiting for its paragraph.
            held_title = para
        else: # This is a regular content paragraph.
            # If we have a title waiting, prepend it to this paragraph and reset.
            if held_title:
                parsed_chunks.append(f"{held_title}\n{para}")
                held_title = "" # Clear the held title as it has been used
            else:
                # This is just a regular paragraph with no title immediately before it.
                parsed_chunks.append(para)

    # After the loop, if there's a title left over (i.e., the very last item was a title),
    # make sure to add it to the final list.
    if held_title:
        parsed_chunks.append(held_title)
    print(parsed_chunks)  
    return "\n\n".join(parsed_chunks)


def inspect_first_filing(data_directory, output_filename="filing_example_titles.txt"):
    """
    Finds the first pickle file, searches for the first row with non-empty text
    sections, and saves that filing's text to a file for inspection.
    """
    print(f"Searching for pickle files in '{data_directory}'...")
    
    # 1. Find the first pickle file automatically
    # ============================================
    pickle_pattern = os.path.join(data_directory, '**', '*.pkl')
    all_pickle_files = glob.glob(pickle_pattern, recursive=True)

    if not all_pickle_files:
        print("Error: No pickle files were found. Please check the directory path.")
        return

    first_file_path = all_pickle_files[0]
    print(f"Found files. Loading the first one: '{os.path.basename(first_file_path)}'")

    # 2. Load the data from the pickle file
    # =======================================
    try:
        with open(first_file_path, 'rb') as f:
            filings_df = pickle.load(f)
            
            if filings_df.empty:
                print("The pickle file contains an empty DataFrame.")
                return

            valid_filing_series = None
            for index, row in filings_df.iterrows():
                risk_text = row.get('rf')
                mda_text = row.get('mgmt')

                if pd.notna(risk_text) and len(str(risk_text).strip()) > 100 and \
                   pd.notna(mda_text) and len(str(mda_text).strip()) > 100:
                    valid_filing_series = row
                    print(f"Found a suitable filing for inspection at index {index}.")
                    break
            
            if valid_filing_series is None:
                print("Error: Could not find a single filing with complete text sections in this file.")
                return

            # 3. Extract data from the VALID filing row
            # ==========================================================
            cik = valid_filing_series.get('cik', 'CIK not found')
            filing_date = valid_filing_series.get('date', 'Date not found')
            risk_factors_text_raw = valid_filing_series.get('rf', 'Section not found.')
            mda_text = valid_filing_series.get('mgmt', 'Section not found.')
            
            # --- Use the NEW, more robust parsing function ---
            risk_factors_text_parsed = parse_risk_factors(risk_factors_text_raw)

            print(f"Extracting data for CIK: {cik}, Filing Date: {filing_date}")


    except Exception as e:
        print(f"An error occurred while loading or processing the pickle file: {e}")
        return

    # 4. Write the extracted text to a plain text file
    # =================================================
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write(f"EXAMPLE FILING FOR INSPECTION\n")
            f.write(f"CIK: {cik} | DATE: {filing_date}\n")
            f.write("="*80 + "\n\n")

            f.write("--- RISK FACTORS (PARSED) ---\n\n")
            f.write(str(risk_factors_text_parsed)) # Write the newly parsed text
            f.write("\n\n")

            f.write("="*80 + "\n\n")

            f.write("--- MANAGEMENT'S DISCUSSION AND ANALYSIS (MD&A) ---\n\n")
            f.write(str(mda_text))
        
        print(f"\nSuccess! The content has been written to '{output_filename}'")
        print("You can now open this file to see the raw text with parsed risk factors.")

    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


# --- How to use this script ---
if __name__ == "__main__":
    # !! IMPORTANT: Make sure this path points to your unzipped text data directory !!
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    
    inspect_first_filing(text_data_path)



Searching for pickle files in 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'...
Found files. Loading the first one: 'text_us_2005.pkl'


C:\Users\carte\AppData\Local\Temp\ipykernel_19912\383267985.py:92: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  filings_df = pickle.load(f)


Found a suitable filing for inspection at index 179123.
['Item 7B Risk Factors.................................................................... 24 ITEM 7B - RISK FACTORS', 'You should carefully consider the risks described below in addition to other information contained or incorporated by reference in this Report before investing in our securities. Realization of any of the following risks could have a material adverse effect on our business, financial condition, cash flows and results of operations.', 'RISKS RELATED TO OUR BUSINESS, STRATEGY AND OPERATIONS\nWE HAVE GENERATED SIGNIFICANT LOSSES IN RECENT PERIODS.', 'We incurred significant net losses during the three years prior to 2004. In 2004, we reported net income of 58.4 million, but we reported net losses of 232.2 million, 56.2 million and 57.2 million for the years ended December 31, 2001, 2002 and 2003, respectively. Our ability to achieve and sustain profitability in the future will depend on the successful continued impl

In [14]:
import os
import glob
import pickle
import pandas as pd

def inspect_multiple_filings(data_directory, num_to_inspect=10, output_filename="filing_examples_raw.txt"):
    """
    Finds and processes a specified number of valid filings and saves their raw,
    unformatted risk factor sections to a single text file for comparison.
    """
    print(f"Searching for pickle files in '{data_directory}'...")
    
    pickle_pattern = os.path.join(data_directory, '**', '*.pkl')
    all_pickle_files = glob.glob(pickle_pattern, recursive=True)

    if not all_pickle_files:
        print("Error: No pickle files were found. Please check the directory path.")
        return

    valid_filings_found = []
    
    # Loop through the pickle files until we find enough examples
    for file_path in all_pickle_files:
        if len(valid_filings_found) >= num_to_inspect:
            break

        print(f"Loading file: '{os.path.basename(file_path)}'")
        try:
            with open(file_path, 'rb') as f:
                filings_df = pickle.load(f)
            
            if filings_df.empty:
                continue # Skip empty dataframes

            # Loop through rows to find filings with complete text
            for index, row in filings_df.iterrows():
                risk_text = row.get('rf')
                mda_text = row.get('mgmt')

                if pd.notna(risk_text) and len(str(risk_text).strip()) > 100 and \
                   pd.notna(mda_text) and len(str(mda_text).strip()) > 100:
                    
                    valid_filings_found.append(row)
                    print(f"  > Found valid filing #{len(valid_filings_found)} at index {index}.")
                    
                    if len(valid_filings_found) >= num_to_inspect:
                        break # Stop searching rows if we have enough
        
        except Exception as e:
            print(f"An error occurred while processing {os.path.basename(file_path)}: {e}")

    if not valid_filings_found:
        print("Could not find any valid filings to inspect.")
        return

    # Now, write all found examples to a single file
    print(f"\nFound {len(valid_filings_found)} valid filings. Writing to '{output_filename}'...")
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            for i, filing_series in enumerate(valid_filings_found):
                cik = filing_series.get('cik', 'N/A')
                filing_date = filing_series.get('date', 'N/A')
                # --- MODIFICATION: Use the raw text directly ---
                risk_factors_raw = filing_series.get('rf', '')
                
                f.write("="*80 + "\n")
                f.write(f"--- EXAMPLE {i+1} of {len(valid_filings_found)} ---\n")
                f.write(f"CIK: {cik} | DATE: {filing_date}\n")
                f.write("="*80 + "\n\n")
                
                # Write the raw, unparsed text to the file
                f.write(str(risk_factors_raw))
                f.write("\n\n\n")

        print("Success! The file has been created with raw, unformatted text.")
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


# --- How to use this script ---
if __name__ == "__main__":
    # !! IMPORTANT: Make sure this path points to your unzipped text data directory !!
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    
    inspect_multiple_filings(text_data_path, num_to_inspect=10)



Searching for pickle files in 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'...
Loading file: 'text_us_2005.pkl'


C:\Users\carte\AppData\Local\Temp\ipykernel_19912\708864471.py:30: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  filings_df = pickle.load(f)


  > Found valid filing #1 at index 179123.
  > Found valid filing #2 at index 179477.
  > Found valid filing #3 at index 179927.
  > Found valid filing #4 at index 180158.
  > Found valid filing #5 at index 180050.
  > Found valid filing #6 at index 180115.
  > Found valid filing #7 at index 180558.
  > Found valid filing #8 at index 180635.
  > Found valid filing #9 at index 180394.
  > Found valid filing #10 at index 181003.

Found 10 valid filings. Writing to 'filing_examples_raw.txt'...
Success! The file has been created with raw, unformatted text.


In [21]:
import os
import glob
import pickle
import pandas as pd
import re # Import the regular expression module

def is_header_heuristic(chunk):
    """
    A new, more precise heuristic based on user findings. A chunk is a header if:
    1. It is a single sentence.
    2. It starts with a capital letter and ends with a period.
    3. It is relatively short (under 25 words).
    This avoids flagging longer, single-sentence content paragraphs as headers.
    """
    if not isinstance(chunk, str):
        return False
        
    stripped_chunk = chunk.strip()

    # Rule 1 & 2: Must be a single sentence starting with a cap and ending with a period.
    # We check if there's exactly one period and it's at the end.
    is_single_sentence = stripped_chunk.count('.') == 1 and stripped_chunk.endswith('.')
    if not is_single_sentence:
        return False

    # Check that it starts with a capital letter.
    if not stripped_chunk[0].isupper():
        return False
        
    # Rule 3: Must be relatively short to be considered a title.
    is_short = len(stripped_chunk.split()) < 40
    if not is_short:
        return False

    # If it passes all checks, it's very likely a title.
    return True

def clean_and_structure_risks(raw_text):
    """
    Intelligently parses raw risk factor text and returns a LIST of structured
    risk items, where each item is a header plus its content.
    """
    if not isinstance(raw_text, str):
        return []

    # 1. Split text into logical chunks based on empty lines.
    chunks = re.split(r'\n\s*\n', raw_text.strip())
    
    # 2. Clean up each chunk and filter out noise.
    cleaned_chunks = []
    for chunk in chunks:
        processed_chunk = ' '.join(chunk.split())
        if processed_chunk and not processed_chunk.isdigit():
            cleaned_chunks.append(processed_chunk)

    # 3. Group chunks into structured risk items (header + content).
    if not cleaned_chunks:
        return []

    structured_items = []
    current_item_chunks = []

    for chunk in cleaned_chunks:
        if is_header_heuristic(chunk):
            if current_item_chunks:
                structured_items.append("\n".join(current_item_chunks))
            current_item_chunks = [chunk]
        else:
            if not current_item_chunks:
                current_item_chunks = [chunk]
            else:
                current_item_chunks.append(chunk)
    
    if current_item_chunks:
        structured_items.append("\n".join(current_item_chunks))

    return structured_items


def inspect_multiple_filings(data_directory, num_to_inspect=10, output_filename="filing_examples_parsed.txt"):
    """
    Finds and processes a specified number of valid filings and saves their
    intelligently parsed risk factor sections to a single text file.
    """
    print(f"Searching for pickle files in '{data_directory}'...")
    
    pickle_pattern = os.path.join(data_directory, '**', '*.pkl')
    all_pickle_files = glob.glob(pickle_pattern, recursive=True)

    if not all_pickle_files:
        print("Error: No pickle files were found. Please check the directory path.")
        return

    valid_filings_found = []
    
    for file_path in all_pickle_files:
        if len(valid_filings_found) >= num_to_inspect: break
        print(f"Loading file: '{os.path.basename(file_path)}'")
        try:
            with open(file_path, 'rb') as f: filings_df = pickle.load(f)
            if filings_df.empty: continue

            for index, row in filings_df.iterrows():
                risk_text = row.get('rf')
                mda_text = row.get('mgmt')

                if pd.notna(risk_text) and len(str(risk_text).strip()) > 100 and \
                   pd.notna(mda_text) and len(str(mda_text).strip()) > 100:
                    valid_filings_found.append(row)
                    print(f"  > Found valid filing #{len(valid_filings_found)} at index {index}.")
                    if len(valid_filings_found) >= num_to_inspect: break
        
        except Exception as e:
            print(f"An error occurred while processing {os.path.basename(file_path)}: {e}")

    if not valid_filings_found:
        print("Could not find any valid filings to inspect.")
        return

    print(f"\nFound {len(valid_filings_found)} valid filings. Writing to '{output_filename}'...")
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            for i, filing_series in enumerate(valid_filings_found):
                cik = filing_series.get('cik', 'N/A')
                filing_date = filing_series.get('date', 'N/A')
                risk_factors_raw = filing_series.get('rf', '')
                
                # This function now returns a list of strings
                risk_items_list = clean_and_structure_risks(risk_factors_raw)
                
                f.write("="*80 + "\n")
                f.write(f"--- EXAMPLE FILING {i+1} of {len(valid_filings_found)} ---\n")
                f.write(f"CIK: {cik} | DATE: {filing_date}\n")
                f.write("="*80 + "\n\n")
                
                if not risk_items_list:
                    f.write("--> No risk chunks could be parsed for this filing.\n\n")
                else:
                    for j, risk_item in enumerate(risk_items_list):
                        f.write(f"---------- Risk Chunk {j+1} ----------\n")
                        f.write(risk_item)
                        f.write("\n\n") # Add extra space after each chunk
                
                f.write("\n\n") # Extra space between filings

        print("Success! The file has been created with clear separators for each risk chunk.")
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


# --- How to use this script ---
if __name__ == "__main__":
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    inspect_multiple_filings(text_data_path, num_to_inspect=10)



Searching for pickle files in 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'...
Loading file: 'text_us_2005.pkl'


C:\Users\carte\AppData\Local\Temp\ipykernel_19912\1484652006.py:100: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  with open(file_path, 'rb') as f: filings_df = pickle.load(f)


  > Found valid filing #1 at index 179123.
  > Found valid filing #2 at index 179477.
  > Found valid filing #3 at index 179927.
  > Found valid filing #4 at index 180158.
  > Found valid filing #5 at index 180050.
  > Found valid filing #6 at index 180115.
  > Found valid filing #7 at index 180558.
  > Found valid filing #8 at index 180635.
  > Found valid filing #9 at index 180394.
  > Found valid filing #10 at index 181003.

Found 10 valid filings. Writing to 'filing_examples_parsed.txt'...
Success! The file has been created with clear separators for each risk chunk.


In [2]:
# This script analyzes stock return data to identify characteristics of top-performing securities.

import pandas as pd
import numpy as np

def analyze_performers(df, percentile, ascending=False):
    """
    Analyzes the characteristics of stocks in a given performance percentile.

    Args:
        df (pd.DataFrame): The input DataFrame with stock data.
        percentile (float): The percentile to analyze (e.g., 0.99 for top 1%).
        ascending (bool): Whether to sort returns ascendingly (for bottom performers).

    Returns:
        pd.DataFrame: A summary DataFrame with analysis results.
    """
    label = "Top" if not ascending else "Bottom"
    print(f"\n--- Analyzing {label} {100 * (1-percentile):.0f}% Performers ---")

    # Determine the return threshold for the given percentile
    return_threshold = df['stock_ret'].quantile(percentile)

    if not ascending:
        subset_df = df[df['stock_ret'] >= return_threshold].copy()
        print(f"Found {len(subset_df)} observations with returns >= {return_threshold:.2%}")
    else:
        # For bottom performers, the percentile is low (e.g., 0.01)
        return_threshold = df['stock_ret'].quantile(1-percentile)
        subset_df = df[df['stock_ret'] <= return_threshold].copy()
        print(f"Found {len(subset_df)} observations with returns <= {return_threshold:.2%}")

    if subset_df.empty:
        print("No data in this performance bracket.")
        return None

    # --- Feature Engineering: Identify Security Type ---
    # Ensure 'id' is a string before using .str accessor
    subset_df['id'] = subset_df['id'].astype(str)
    
    conditions = [
        subset_df['id'].str.endswith('W'),
        subset_df['id'].str.endswith('C')
    ]
    choices = ['Warrant', 'Common Stock']
    subset_df['security_type'] = np.select(conditions, choices, default='Other')

    # --- Group and Analyze ---
    # Define the characteristics we want to analyze by taking their mean
    analysis_cols = {
        'stock_ret': 'mean',
        'age': 'mean',           # Company age
        'qmj': 'mean',           # Quality-Minus-Junk factor
        'betadown_252d': 'mean', # Downside Beta (risk)
        'prc_highprc_252d': 'mean' # Price vs 1-year high (momentum)
    }
    
    # Add a column for counting observations in each group
    subset_df['count'] = 1
    analysis_cols['count'] = 'sum'


    # Group by country and security type and aggregate
    summary = subset_df.groupby(['excntry', 'security_type']).agg(analysis_cols)
    
    # Make the count an integer
    summary['count'] = summary['count'].astype(int)

    return summary.sort_values(by='count', ascending=False)


# --- Main Execution Block ---
if __name__ == "__main__":
    file_path = r"C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.parquet"
    print(f"--- Loading data from '{file_path}' ---")

    try:
        df = pd.read_parquet(file_path)
        print("Data loaded successfully.")

        # Set pandas display options for better readability
        pd.set_option('display.width', 120)
        pd.set_option('display.max_rows', 100)

        # Analyze the top 1% of performers
        top_performers_summary = analyze_performers(df, 0.99, ascending=False)
        if top_performers_summary is not None:
            print("\n--- Summary of Top 1% Performers (Grouped by Country & Type) ---")
            print(top_performers_summary)

        # You could also analyze the bottom 1% for comparison
        bottom_performers_summary = analyze_performers(df, 0.99, ascending=True)
        if bottom_performers_summary is not None:
            print("\n--- Summary of Bottom 1% Performers (Grouped by Country & Type) ---")
            print(bottom_performers_summary)


    except FileNotFoundError:
        print(f"Error: The file was not found at '{file_path}'")
    except KeyError as e:
        print(f"Error: A required column is missing from the Parquet file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


--- Loading data from 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.parquet' ---
Data loaded successfully.

--- Analyzing Top 1% Performers ---
Found 64015 observations with returns >= 59.14%

--- Summary of Top 1% Performers (Grouped by Country & Type) ---
                        stock_ret         age       qmj  betadown_252d  prc_highprc_252d  count
excntry security_type                                                                          
USA     Other            2.699218  177.113595 -0.841639       1.076764          0.412847  10467
AUS     Warrant          1.101760  183.681911 -0.536992       1.098127          0.486230   9796
HKG     Warrant          1.113656  289.557014 -0.477659       0.878736          0.580091   6323
CAN     Common Stock     1.125791  152.998500 -0.721148       1.101458          0.499535   5334
KOR     Warrant          0.941709  283.945376 -0.405381       1.014208          0.618231   5309
GBR     Warrant          1.457628  193.445761 -0.617220  

In [4]:
# This script loads stock data, analyzes the country distribution for a specific list of IDs,
# and compares it to the country distribution of the entire dataset.

import pandas as pd
import numpy as np

# --- Main Execution Block ---
if __name__ == "__main__":
    file_path = r"C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.parquet"
    print(f"--- Loading data from '{file_path}' ---")

    # Define the list of specific IDs to look up from your images
    ids_to_find = [
        # Top Performers
        'comp_278548_01W', 'comp_208686_02W', 'comp_240865_01W', 
        'comp_275020_01W', 'comp_166669_01C', 'comp_277578_01W', 
        'comp_065840_01C', 'comp_356061_01W', 'comp_210323_01W', 
        'comp_026353_01C',
        # Bottom Performers
        'crsp_19649', 'comp_321951_01W', 'crsp_19136', 'comp_314949_01W',
        'crsp_21280', 'crsp_23267', 'crsp_13556', 'crsp_15375',
        'crsp_12350', 'comp_298295_01W'
    ]

    try:
        df = pd.read_parquet(file_path)
        print("Data loaded successfully.")

        # Ensure the 'id' column is of string type for matching
        df['id'] = df['id'].astype(str)

        # --- Filter the DataFrame to find the specified IDs ---
        print(f"\n--- Analyzing country distribution for {len(ids_to_find)} specific IDs... ---")
        
        results_df = df[df['id'].isin(ids_to_find)]

        if not results_df.empty:
            # --- 1. Calculate proportion for the sample list of unique stocks ---
            # We drop duplicates to count each stock only once
            unique_sample_stocks = results_df.drop_duplicates(subset=['id'])
            sample_proportions = unique_sample_stocks['excntry'].value_counts(normalize=True).rename('sample_proportion')

            # --- 2. Calculate proportion for all unique stocks in the entire dataset ---
            # This provides a fair baseline for comparison
            unique_total_stocks = df.drop_duplicates(subset=['id'])
            overall_proportions = unique_total_stocks['excntry'].value_counts(normalize=True).rename('overall_proportion')

            # --- 3. Combine, compare, and display the two distributions ---
            comparison_df = pd.concat([sample_proportions, overall_proportions], axis=1).fillna(0)
            
            # Calculate an "enrichment" factor to see how over/under-represented each country is
            comparison_df['enrichment'] = (comparison_df['sample_proportion'] / comparison_df['overall_proportion']).replace([np.inf, -np.inf], 0)

            # Format the columns for better readability
            comparison_df['sample_proportion'] = comparison_df['sample_proportion'].map('{:.2%}'.format)
            comparison_df['overall_proportion'] = comparison_df['overall_proportion'].map('{:.2%}'.format)
            comparison_df['enrichment'] = comparison_df['enrichment'].map('{:.2f}x'.format)

            print("\n--- Comparison of Country Proportions ---")
            print("(Sample vs. Entire Dataset of Unique Stocks)")
            print(comparison_df.sort_values(by='sample_proportion', ascending=False))
            
        else:
            print("Could not find any of the specified IDs in the dataset.")

    except FileNotFoundError:
        print(f"Error: The file was not found at '{file_path}'")
    except KeyError as e:
        print(f"Error: A required column is missing from the Parquet file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")



--- Loading data from 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.parquet' ---
Data loaded successfully.

--- Analyzing country distribution for 20 specific IDs... ---

--- Comparison of Country Proportions ---
(Sample vs. Entire Dataset of Unique Stocks)
        sample_proportion overall_proportion enrichment
excntry                                                
SGP                 5.00%              1.94%      2.58x
IRL                 5.00%              0.14%     35.64x
USA                35.00%             25.58%      1.37x
DEU                25.00%              3.15%      7.94x
CAN                15.00%              4.53%      3.31x
FRA                15.00%              2.69%      5.59x
ESP                 0.00%              0.79%      0.00x
PRT                 0.00%              0.16%      0.00x
AUT                 0.00%              0.27%      0.00x
MEX                 0.00%              0.37%      0.00x
NZL                 0.00%              0.42%      0.00x
F

In [1]:
import pandas as pd
import numpy as np
import os

# Define the path to the Parquet file
pq_path = r'C:\_Files\School\Competitions\FIAM2025\data\quant_data\ret_sample.parquet'

try:
    # Read the Parquet file into a DataFrame
    df = pd.read_parquet(pq_path)

    # Define the target column
    column_name = 'ret_eom'

    if column_name in df.columns:
        # Calculate the number of empty (NaN) values
        missing_count = df[column_name].isna().sum()
        
        # Calculate the total number of rows
        total_count = len(df[column_name])
        
        # Calculate the percentage of empty values
        if total_count > 0:
            percent_empty = (missing_count / total_count) * 100
            print(f"--- Analysis for column: '{column_name}' ---")
            print(f"Total rows: {total_count:,}")
            print(f"Empty rows: {missing_count:,}")
            print(f"Percent empty: {percent_empty:.2f}%")
        else:
            print(f"The DataFrame is empty. Cannot calculate percentage.")
            
    else:
        print(f"Error: Column '{column_name}' not found in the Parquet file.")
        print("Available columns are:")
        print(df.columns.tolist())

except Exception as e:
    print(f"An error occurred: {e}")

--- Analysis for column: 'ret_eom' ---
Total rows: 6,100,030
Empty rows: 0
Percent empty: 0.00%


In [5]:
import pandas as pd 

text = pd.read_pickle(r'C:\_Files\School\Competitions\FIAM2025\data\global_text_data\SS.pkl')

print(text.head)


<bound method NDFrame.head of                   document_id  \
0        FF-QR3-2023-SBB B-SS   
1        FF-QR3-2019-ATORX-SS   
2           FF-AR-2022-LUG-SS   
3         FF-AR-2020-SINCH-SS   
4     FF-QR1-2022-VOLCAR B-SS   
...                       ...   
5224     FF-QR1-2020-HAV B-SS   
5225     FF-QR4-2015-TOBII-SS   
5226       FF-QR1-2024-AZA-SS   
5227      FF-QR3-2015-VIVE-SS   
5228   FF-QR3-2022-ATRLJ B-SS   

           section_5_management_discussion_and_analysis  \
0     SBB Interim Report, January–September 2023 Pag...   
1      activities. Partnering/out-licensing Initial ...   
2      2022 ANNUAL REPORT Dear Fellow Shareholders, ...   
3     tober, Sinch launched Sinch Conversation API, ...   
4     uity instruments in Luminar which are listed, ...   
...                                                 ...   
5224  nedskrivning av Avvecklingsportföljens framtid...   
5225  Tobii Tech continues as planned to invest in t...   
5226  rim Report January–March 2024, Avanz

In [2]:
import os
import pandas as pd
import glob
import logging

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- !! IMPORTANT: SET THESE PATHS !! ---
# 1. The directory containing your exchange pickle files
#    (e.g., r'D:\market_data\text_data\WORLD_EXCHANGES')
PICKLE_BASE_DIR = r'C:\_Files\School\Competitions\FIAM2025\data\global_text_data'

# 2. The exact name of the country column in your DataFrames
COUNTRY_COLUMN = 'country' 

# --- Script ---

def discover_countries(pickle_dir):
    """
    Scans all .pkl files in the specified directory to find and print
    a unique list of all country names.
    """
    logging.info("--- Starting Part 1: Country Discovery ---")
    if PICKLE_BASE_DIR == 'PLEASE_SET_THIS_PATH':
        logging.error("Please update the 'PICKLE_BASE_DIR' variable in this script.")
        return

    logging.info(f"Scanning for .pkl files in: {pickle_dir}")
    
    # Find all .pkl files. 
    # Use '*.pkl' for a flat directory.
    # Use '**/*.pkl' (and set recursive=True) if they are in subfolders.
    try:
        pickle_files = glob.glob(os.path.join(pickle_dir, '*.pkl'), recursive=False)
    except Exception as e:
        logging.error(f"Error finding pickle files: {e}")
        return

    if not pickle_files:
        logging.warning(f"No .pkl files found in {pickle_dir}. Check the path.")
        return
        
    logging.info(f"Found {len(pickle_files)} pickle files to scan.")
    
    all_countries_set = set()

    for pkl_file in pickle_files:
        try:
            logging.info(f"Scanning: {os.path.basename(pkl_file)}")
            df = pd.read_pickle(pkl_file)
            
            if COUNTRY_COLUMN not in df.columns:
                logging.warning(f"  Skipping... '{COUNTRY_COLUMN}' not found in this file.")
                continue
                
            # Get all unique, non-null country names from the column
            unique_in_file = df[COUNTRY_COLUMN].dropna().unique()
            
            if len(unique_in_file) > 0:
                # Add the new countries to our master set
                all_countries_set.update(unique_in_file)
                
        except Exception as e:
            logging.error(f"  Error reading {pkl_file}: {e}", exc_info=False)

    # --- Finished ---
    logging.info("--- Discovery Complete ---")
    
    if not all_countries_set:
        logging.error("No countries found. Please check your data and configuration.")
        return
        
    sorted_countries = sorted(list(all_countries_set))
    
    logging.info(f"Found {len(sorted_countries)} unique countries across all files:")
    print("\n--- Unique Country List ---")
    for country in sorted_countries:
        print(country)
    print("----------------------------\n")
    logging.info("Please use this list to build your 'COUNTRY_TO_REGION_MAP' in script 2.")

if __name__ == "__main__":
    discover_countries(PICKLE_BASE_DIR)


2025-10-30 17:42:01,729 - INFO - --- Starting Part 1: Country Discovery ---
2025-10-30 17:42:01,729 - INFO - Scanning for .pkl files in: C:\_Files\School\Competitions\FIAM2025\data\global_text_data
2025-10-30 17:42:01,730 - INFO - Found 26 pickle files to scan.
2025-10-30 17:42:01,731 - INFO - Scanning: AT.pkl
2025-10-30 17:42:02,298 - INFO - Scanning: AV.pkl
2025-10-30 17:42:02,420 - INFO - Scanning: BB.pkl
2025-10-30 17:42:02,539 - INFO - Scanning: CG.pkl
2025-10-30 17:42:03,696 - INFO - Scanning: CT.pkl
2025-10-30 17:42:04,544 - INFO - Scanning: DC.pkl
2025-10-30 17:42:04,722 - INFO - Scanning: FH.pkl
2025-10-30 17:42:04,806 - INFO - Scanning: FP.pkl
2025-10-30 17:42:05,091 - INFO - Scanning: GF.pkl
2025-10-30 17:42:05,567 - INFO - Scanning: HK.pkl
2025-10-30 17:42:09,135 - INFO - Scanning: ID.pkl
2025-10-30 17:42:10,132 - INFO - Scanning: IM.pkl
2025-10-30 17:42:10,201 - INFO - Scanning: IT.pkl
2025-10-30 17:42:10,246 - INFO - Scanning: JT.pkl
2025-10-30 17:42:10,319 - INFO - Scann


--- Unique Country List ---
Australia
Austria
Belgium
Canada
China
Denmark
Finland
France
Germany
Hong Kong
Ireland
Israel
Italy
Japan
Luxembourg
Mexico
Netherlands
Norway
Portugal
Singapore
South Korea
Spain
Sweden
Switzerland
Taiwan
United Kingdom
----------------------------



In [6]:
import os
import pandas as pd
import glob
import logging
import time
from collections import defaultdict

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


PICKLE_BASE_DIR = r'C:\_Files\School\Competitions\FIAM2025\data\global_text_data' 


OUTPUT_DIR = r'D:\market_data\text_data\REGIONS'

# --- !! IMPORTANT: DEFINE YOUR MAP !! ---
# 3. Fill this map based on the output from script 1.
#    Any country NOT in this map will be SKIPPED.
COUNTRY_TO_REGION_MAP = {
    # 'Country Name': 'REGION_NAME'
    
    # --- CANADA ---
    'Canada': 'CANADA',
    
    # --- NORDICS ---
    'Denmark': 'NORDICS',
    'Finland': 'NORDICS',
    'Norway': 'NORDICS',
    'Sweden': 'NORDICS',
    
    # --- ASIA_PACIFIC ---
    'China': 'ASIA_PACIFIC',
    'Hong Kong': 'ASIA_PACIFIC',
    'Japan': 'ASIA_PACIFIC',
    'Singapore': 'ASIA_PACIFIC',
    'South Korea': 'ASIA_PACIFIC',
    'Taiwan': 'ASIA_PACIFIC',
    
    # --- GERMANIC MAINLAND ---
    'Germany': 'GERMANIC_MAINLAND',
    'Austria': 'GERMANIC_MAINLAND',
    'Switzerland': 'GERMANIC_MAINLAND',
    'Netherlands': 'GERMANIC_MAINLAND',
    'Belgium': 'GERMANIC_MAINLAND',
    'Luxembourg': 'GERMANIC_MAINLAND',

    # --- SOUTHERN EUROPE ---
    'France': 'SOUTHERN_EUROPE',
    'Italy': 'SOUTHERN_EUROPE',
    'Spain': 'SOUTHERN_EUROPE',
    'Portugal': 'SOUTHERN_EUROPE',
    
    # --- ANGLOSPHERE / COMMON LAW ---
    'United Kingdom': 'ANGLOSPHERE_COMMONLAW',
    'Ireland': 'ANGLOSPHERE_COMMONLAW',
    'Australia': 'ANGLOSPHERE_COMMONLAW',
    
    # --- LATIN AMERICA ---
    'Mexico': 'LATAM',
    
    # --- MIDDLE EAST ---
    'Israel': 'MIDDLE_EAST',
}

# --- Columns & Filters ---
COUNTRY_COLUMN = 'country'
TEXT_COLUMNS = ['section_7_risk_factors', 'section_5_management_discussion_and_analysis'] # Text columns to extract
MIN_TEXT_LENGTH = 50 # Minimum characters for a text section to be included
LOG_INTERVAL = 5000 # Log progress every N filings processed

# --- Script ---

def build_corpora(pickle_dir, output_dir, region_map):
    """
    Scans all .pkl files, maps rows to regions, and appends text
    to the correct regional corpus file.
    """
    logging.info("--- Starting Part 2: Regional Corpus Build ---")
    if PICKLE_BASE_DIR == 'PLEASE_SET_THIS_PATH' or OUTPUT_DIR == 'PLEASE_SET_THIS_PATH':
        logging.error("Please update the 'PICKLE_BASE_DIR' and 'OUTPUT_DIR' variables.")
        return
    if not region_map:
        logging.error("The 'COUNTRY_TO_REGION_MAP' is empty. Please define your regions.")
        return

    os.makedirs(output_dir, exist_ok=True)
    
    # --- Open all output files ---
    # Get all unique region names from your map
    all_regions = set(region_map.values())
    output_files = {} # Dictionary to hold the open file handles
    
    logging.info(f"Opening {len(all_regions)} output files...")
    for region_name in all_regions:
        try:
            output_path = os.path.join(output_dir, f"{region_name}_corpus.txt")
            # Open in 'append' mode to allow restarting
            file_handle = open(output_path, 'a', encoding='utf-8')
            output_files[region_name] = file_handle
            logging.info(f"Opened {output_path} for writing.")
        except Exception as e:
            logging.error(f"Failed to open output file for {region_name}: {e}")
            
    if not output_files:
        logging.error("No output files could be opened. Exiting.")
        return

    # --- Scan all pickle files ---
    logging.info(f"Scanning for .pkl files in: {pickle_dir}")
    pickle_files = glob.glob(os.path.join(pickle_dir, '*.pkl'), recursive=False)
    
    if not pickle_files:
        logging.warning(f"No .pkl files found in {pickle_dir}.")
        return

    logging.info(f"Found {len(pickle_files)} pickle files to process.")
    
    start_time = time.time()
    total_rows_processed = 0
    total_segments_written = 0
    unmapped_rows = 0
    stats = defaultdict(int) # Track segments per region

    try:
        for pkl_file in pickle_files:
            logging.info(f"Processing: {os.path.basename(pkl_file)}")
            try:
                df = pd.read_pickle(pkl_file)
                
                # Check for required columns
                required_cols = TEXT_COLUMNS + [COUNTRY_COLUMN]
                missing_cols = [col for col in required_cols if col not in df.columns]
                if missing_cols:
                    logging.warning(f"  Skipping... Missing columns: {missing_cols}")
                    continue

                # Iterate through rows
                for index, row in df.iterrows():
                    total_rows_processed += 1
                    
                    country = row[COUNTRY_COLUMN]
                    # Find the region for this country from your map
                    region = region_map.get(country)
                    
                    if not region:
                        unmapped_rows += 1
                        continue # Skip this row if country is not in our map
                        
                    # Get the correct output file handle
                    outfile = output_files.get(region)
                    if not outfile:
                        logging.warning(f"No output file handle for region '{region}'. Skipping.")
                        continue
                        
                    # Extract text from the text columns
                    for col in TEXT_COLUMNS:
                        text_content = row[col]
                        if isinstance(text_content, str) and len(text_content.strip()) >= MIN_TEXT_LENGTH:
                            cleaned_text = text_content.strip()
                            outfile.write(cleaned_text + '\n\n')
                            total_segments_written += 1
                            stats[region] += 1
                            
                    if total_rows_processed % LOG_INTERVAL == 0:
                        elapsed = time.time() - start_time
                        rate = total_rows_processed / elapsed if elapsed > 0 else 0
                        logging.info(f"  Processed {total_rows_processed:,} total rows | Written {total_segments_written:,} segments | Rate: {rate:.1f} rows/s")
                        
            except Exception as e:
                logging.error(f" D-Error reading {pkl_file}: {e}", exc_info=False)
                
    finally:
        # --- Clean up ---
        logging.info("--- Processing Complete. Closing all files. ---")
        for region, handle in output_files.items():
            handle.close()
            logging.info(f"Closed {region} file.")
            
    end_time = time.time()
    total_time = end_time - start_time
    logging.info(f"Total rows scanned: {total_rows_processed:,}")
    logging.info(f"Total rows skipped (unmapped country): {unmapped_rows:,}")
    logging.info(f"Total text segments written: {total_segments_written:,}")
    logging.info(f"Total time: {total_time:.2f} seconds")
    
    print("\n--- Segments Written per Region ---")
    for region, count in stats.items():
        print(f"{region}: {count:,}")
    print("---------------------------------")


if __name__ == "__main__":
    build_corpora(PICKLE_BASE_DIR, OUTPUT_DIR, COUNTRY_TO_REGION_MAP)


2025-10-30 18:04:41,363 - INFO - --- Starting Part 2: Regional Corpus Build ---
2025-10-30 18:04:41,364 - INFO - Opening 8 output files...
2025-10-30 18:04:41,365 - INFO - Opened D:\market_data\text_data\REGIONS\GERMANIC_MAINLAND_corpus.txt for writing.
2025-10-30 18:04:41,365 - INFO - Opened D:\market_data\text_data\REGIONS\LATAM_corpus.txt for writing.
2025-10-30 18:04:41,366 - INFO - Opened D:\market_data\text_data\REGIONS\MIDDLE_EAST_corpus.txt for writing.
2025-10-30 18:04:41,366 - INFO - Opened D:\market_data\text_data\REGIONS\ASIA_PACIFIC_corpus.txt for writing.
2025-10-30 18:04:41,367 - INFO - Opened D:\market_data\text_data\REGIONS\CANADA_corpus.txt for writing.
2025-10-30 18:04:41,368 - INFO - Opened D:\market_data\text_data\REGIONS\ANGLOSPHERE_COMMONLAW_corpus.txt for writing.
2025-10-30 18:04:41,368 - INFO - Opened D:\market_data\text_data\REGIONS\SOUTHERN_EUROPE_corpus.txt for writing.
2025-10-30 18:04:41,369 - INFO - Opened D:\market_data\text_data\REGIONS\NORDICS_corpus.


--- Segments Written per Region ---
ANGLOSPHERE_COMMONLAW: 40,243
GERMANIC_MAINLAND: 20,166
ASIA_PACIFIC: 153,581
CANADA: 25,698
NORDICS: 14,195
SOUTHERN_EUROPE: 13,622
MIDDLE_EAST: 2,347
LATAM: 442
---------------------------------


In [8]:
import os
import logging

# --- Configuration ---

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Base output directory (MUST MATCH your scraper script)
# !! IMPORTANT: Update this path !!
BASE_OUTPUT_DIR = r'D:\market_data\text_data' # <-- MAKE SURE THIS PATH IS CORRECT
USA_OUTPUT_DIR = os.path.join(BASE_OUTPUT_DIR, 'USA')
USA_CORPUS_FILE = os.path.join(USA_OUTPUT_DIR, 'corpus_cleaned.txt')

# How many bytes to read from the end (approx 5000 characters)
# Adjust if needed. 1 char ~ 1 byte in UTF-8 for basic English text.
BYTES_TO_READ = 1000000 # Read slightly more to account for multi-byte chars or partial lines

# --- Script ---

def inspect_tail(filepath, num_bytes):
    """Reads and prints the last `num_bytes` of a file."""
    logging.info(f"Attempting to read the last ~{num_bytes} bytes from: {filepath}")
    try:
        with open(filepath, 'rb') as f: # Open in binary read mode ('rb')
            f.seek(0, os.SEEK_END) # Go to the end of the file
            filesize = f.tell()
            logging.info(f"File size: {filesize:,} bytes")

            if filesize == 0:
                logging.warning("File is empty.")
                return

            seek_position = max(0, filesize - num_bytes) # Calculate position, ensure non-negative
            f.seek(seek_position, os.SEEK_SET) # Jump near the end

            last_chunk = f.read() # Read from seek position to the end

            # Decode the bytes into text, ignoring errors for potentially cut-off characters at the beginning
            # Using UTF-8, which is what the scraper saves in
            decoded_text = last_chunk.decode('utf-8', errors='ignore')

            print("\n--- Last ~5000 Characters ---")
            # Print the decoded text. It might start mid-word/mid-sentence, which is expected.
            print(decoded_text)
            print("--- End of File Sample ---")

    except FileNotFoundError:
        logging.error(f"Error: File not found at {filepath}")
    except Exception as e:
        logging.error(f"An error occurred while reading the file: {e}", exc_info=True)

if __name__ == "__main__":
    if not os.path.exists(USA_CORPUS_FILE):
        logging.error(f"Corpus file does not exist: {USA_CORPUS_FILE}")
        logging.error("Please ensure the BASE_OUTPUT_DIR is correct and the scraper has run.")
    else:
        inspect_tail(USA_CORPUS_FILE, BYTES_TO_READ)

2025-10-27 08:42:05,815 - INFO - Attempting to read the last ~1000000 bytes from: D:\market_data\text_data\USA\corpus_cleaned.txt
2025-10-27 08:42:05,817 - INFO - File size: 12,334,834,157 bytes



--- Last ~5000 Characters ---
5 to Buy and 2 to Hold. Shares are priced at $28.16 and have an average target of $49.14, giving the stock a 46% upside for the year ahead. (See VMEO stock analysis on TipRanks)
Boyd Gaming (BYD)
Next up is Boyd Gaming, a major operator of casino gaming properties. The company got its start in 1975, in Las Vegas, and has since spread out of its hometown. Boyd is now active in 10 states (Illinois, Indiana, Iowa, Kansas, Louisiana, Mississippi, Missouri, Nevada, Ohio, and Pennsylvania), and boasts 28 casino facilities.
Like much of the leisure and entertainment industry, Boyd was badly hurt by the corona crisis in 2020. The company was forced to announce mass layoffs – 25% of its workforce – in July of last year, and saw its 2Q20 revenues fall to just $209 million, approximately one-quarter normal levels. The company is experiencing better times this year, however, as the economy reopens and consumers begin spending on much-needed leisure.
In its 2Q21 repor

In [6]:
import os

BASE_OUTPUT_DIR = r'D:\market_data\text_data'
USA_OUTPUT_DIR = os.path.join(BASE_OUTPUT_DIR, 'USA')
USA_CORPUS_FILE = os.path.join(USA_OUTPUT_DIR, 'corpus.txt')

count = 0
chunk_size = 1024 * 1024 * 64  # 64 MB chunks
buffer = ""

with open(USA_CORPUS_FILE, 'r', encoding='utf-8', errors='ignore') as f:
    while True:
        chunk = f.read(chunk_size)
        if not chunk:
            break
        # Prepend the last few chars from the previous buffer
        chunk = buffer + chunk

        # Count occurrences
        count += chunk.count('\n\n')

        # Keep only the last character in case '\n' is split across chunks
        buffer = chunk[-1:]

print(count)


8090090


In [ ]:
data = {
    
{'loss': 1.4536, 'grad_norm': 6.79431676864624, 'learning_rate': 4.746605301545849e-05, 'epoch': 0.15},
{'loss': 1.4583, 'grad_norm': 6.5994462966918945, 'learning_rate': 4.74633017098264e-05, 'epoch': 0.15},
{'loss': 1.4758, 'grad_norm': 7.309181213378906, 'learning_rate': 4.746055040419432e-05, 'epoch': 0.15},
{'loss': 1.4782, 'grad_norm': 7.764740943908691, 'learning_rate': 4.745779909856222e-05, 'epoch': 0.15},
{'loss': 1.4527, 'grad_norm': 10.764654159545898, 'learning_rate': 4.7455047792930135e-05, 'epoch': 0.15},
{'loss': 1.4587, 'grad_norm': 11.2777681350708, 'learning_rate': 4.745229648729805e-05, 'epoch': 0.15},
{'loss': 1.4756, 'grad_norm': 10.040223121643066, 'learning_rate': 4.744954518166596e-05, 'epoch': 0.15},
{'loss': 1.4681, 'grad_norm': 7.98757266998291, 'learning_rate': 4.7446793876033873e-05, 'epoch': 0.15},
{'loss': 1.4635, 'grad_norm': 21.996435165405273, 'learning_rate': 4.744404257040179e-05, 'epoch': 0.15},
{'loss': 1.4822, 'grad_norm': 7.241261959075928, 'learning_rate': 4.74412912647697e-05, 'epoch': 0.15},
{'loss': 1.4739, 'grad_norm': 8.585515975952148, 'learning_rate': 4.743853995913761e-05, 'epoch': 0.15},
{'loss': 1.4781, 'grad_norm': 8.846548080444336, 'learning_rate': 4.743578865350553e-05, 'epoch': 0.15},
{'loss': 1.4857, 'grad_norm': 8.616715431213379, 'learning_rate': 4.7433037347873436e-05, 'epoch': 0.15},
{'loss': 1.4869, 'grad_norm': 11.389678955078125, 'learning_rate': 4.7430286042241344e-05, 'epoch': 0.15},
{'loss': 1.4758, 'grad_norm': 8.24552059173584, 'learning_rate': 4.742753473660926e-05, 'epoch': 0.15},
{'loss': 1.4808, 'grad_norm': 6.947010517120361, 'learning_rate': 4.7424783430977174e-05, 'epoch': 0.15},
{'loss': 1.4833, 'grad_norm': 49.08424377441406, 'learning_rate': 4.742203212534508e-05, 'epoch': 0.15},
{'loss': 1.4528, 'grad_norm': 8.930402755737305, 'learning_rate': 4.7419280819713e-05, 'epoch': 0.15},
{'loss': 1.4744, 'grad_norm': 10.954212188720703, 'learning_rate': 4.741652951408091e-05, 'epoch': 0.16},
{'loss': 1.4728, 'grad_norm': 16.19570541381836, 'learning_rate': 4.741377820844882e-05, 'epoch': 0.16},
{'loss': 1.4806, 'grad_norm': 9.774336814880371, 'learning_rate': 4.7411026902816736e-05, 'epoch': 0.16},
{'loss': 1.4755, 'grad_norm': 12.006851196289062, 'learning_rate': 4.740827559718465e-05, 'epoch': 0.16},
{'loss': 1.4613, 'grad_norm': 10.727494239807129, 'learning_rate': 4.740552429155256e-05, 'epoch': 0.16},
{'loss': 1.48, 'grad_norm': 10.045310974121094, 'learning_rate': 4.740277298592047e-05, 'epoch': 0.16},
{'loss': 1.4812, 'grad_norm': 9.043827056884766, 'learning_rate': 4.7400021680288383e-05, 'epoch': 0.16},
{'loss': 1.4542, 'grad_norm': 13.474766731262207, 'learning_rate': 4.739727037465629e-05, 'epoch': 0.16}
{'loss': 1.4962, 'grad_norm': 9.363969802856445, 'learning_rate': 4.739451906902421e-05, 'epoch': 0.16}
{'loss': 1.4913, 'grad_norm': 9.69883918762207, 'learning_rate': 4.739176776339212e-05, 'epoch': 0.16}
{'loss': 1.4841, 'grad_norm': 9.489829063415527, 'learning_rate': 4.738901645776003e-05, 'epoch': 0.16}
{'loss': 1.4769, 'grad_norm': 11.249781608581543, 'learning_rate': 4.7386265152127946e-05, 'epoch': 0.16}
{'loss': 1.4693, 'grad_norm': 10.347302436828613, 'learning_rate': 4.738351384649586e-05, 'epoch': 0.16}
{'loss': 1.4778, 'grad_norm': 9.758716583251953, 'learning_rate': 4.738076254086377e-05, 'epoch': 0.16}
{'loss': 1.4737, 'grad_norm': 6.0282883644104, 'learning_rate': 4.7378011235231684e-05, 'epoch': 0.16}
{'loss': 1.4815, 'grad_norm': 8.643651008605957, 'learning_rate': 4.737525992959959e-05, 'epoch': 0.16}
{'loss': 1.475, 'grad_norm': 7.774406909942627, 'learning_rate': 4.73725086239675e-05, 'epoch': 0.16}
{'loss': 1.4688, 'grad_norm': 11.211589813232422, 'learning_rate': 4.7369757318335416e-05, 'epoch': 0.16}
{'loss': 1.4721, 'grad_norm': 7.872531414031982, 'learning_rate': 4.736700601270333e-05, 'epoch': 0.16}
{'loss': 1.4941, 'grad_norm': 10.49804401397705, 'learning_rate': 4.736425470707124e-05, 'epoch': 0.16}
{'loss': 1.4732, 'grad_norm': 8.337812423706055, 'learning_rate': 4.7361503401439155e-05, 'epoch': 0.16}
{'loss': 1.4987, 'grad_norm': 9.663531303405762, 'learning_rate': 4.735875209580707e-05, 'epoch': 0.16}
{'loss': 1.4932, 'grad_norm': 7.8786396980285645, 'learning_rate': 4.735600079017498e-05, 'epoch': 0.16}
{'loss': 1.4706, 'grad_norm': 9.960122108459473, 'learning_rate': 4.7353249484542893e-05, 'epoch': 0.16}
{'loss': 1.4616, 'grad_norm': 7.0666728019714355, 'learning_rate': 4.735049817891081e-05, 'epoch': 0.16}
{'loss': 1.4764, 'grad_norm': 10.718029022216797, 'learning_rate': 4.734774687327872e-05, 'epoch': 0.16}
{'loss': 1.4838, 'grad_norm': 11.897994995117188, 'learning_rate': 4.7344995567646625e-05, 'epoch': 0.16}
{'loss': 1.4751, 'grad_norm': 13.172527313232422, 'learning_rate': 4.734224426201454e-05, 'epoch': 0.16}
{'loss': 1.4572, 'grad_norm': 6.765641689300537, 'learning_rate': 4.7339492956382456e-05, 'epoch': 0.16}
{'loss': 1.4548, 'grad_norm': 8.282583236694336, 'learning_rate': 4.7336741650750364e-05, 'epoch': 0.16}
{'loss': 1.4794, 'grad_norm': 10.229754447937012, 'learning_rate': 4.733399034511828e-05, 'epoch': 0.16}
{'loss': 1.486, 'grad_norm': 11.479272842407227, 'learning_rate': 4.7331239039486194e-05, 'epoch': 0.16}
{'loss': 1.4816, 'grad_norm': 6.574367523193359, 'learning_rate': 4.73284877338541e-05, 'epoch': 0.16}
{'loss': 1.475, 'grad_norm': 8.249167442321777, 'learning_rate': 4.732573642822202e-05, 'epoch': 0.16}
{'loss': 1.4763, 'grad_norm': 11.061763763427734, 'learning_rate': 4.7322985122589926e-05, 'epoch': 0.16}
{'loss': 1.4733, 'grad_norm': 8.99709701538086, 'learning_rate': 4.7320233816957835e-05, 'epoch': 0.16}
{'loss': 1.4485, 'grad_norm': 9.03708553314209, 'learning_rate': 4.731748251132575e-05, 'epoch': 0.16}
{'loss': 1.4789, 'grad_norm': 7.607971668243408, 'learning_rate': 4.7314731205693665e-05, 'epoch': 0.16}
{'loss': 1.4816, 'grad_norm': 10.690608978271484, 'learning_rate': 4.731197990006157e-05, 'epoch': 0.16}
{'loss': 1.5158, 'grad_norm': 7.525801181793213, 'learning_rate': 4.730922859442949e-05, 'epoch': 0.16}
{'loss': 1.4607, 'grad_norm': 6.880888938903809, 'learning_rate': 4.7306477288797404e-05, 'epoch': 0.16}
{'loss': 1.4676, 'grad_norm': 7.290761947631836, 'learning_rate': 4.730372598316531e-05, 'epoch': 0.16}
{'loss': 1.5065, 'grad_norm': 9.399045944213867, 'learning_rate': 4.730097467753323e-05, 'epoch': 0.16}
{'loss': 1.4745, 'grad_norm': 8.392634391784668, 'learning_rate': 4.729822337190114e-05, 'epoch': 0.16}
{'loss': 1.49, 'grad_norm': 9.712846755981445, 'learning_rate': 4.729547206626905e-05, 'epoch': 0.16}
{'loss': 1.4738, 'grad_norm': 8.620290756225586, 'learning_rate': 4.729272076063696e-05, 'epoch': 0.16}
{'loss': 1.4788, 'grad_norm': 6.871674060821533, 'learning_rate': 4.7289969455004874e-05, 'epoch': 0.16}
{'loss': 1.4816, 'grad_norm': 8.319815635681152, 'learning_rate': 4.728721814937279e-05, 'epoch': 0.16}
{'loss': 1.4936, 'grad_norm': 11.031319618225098, 'learning_rate': 4.72844668437407e-05, 'epoch': 0.16}
{'loss': 1.4819, 'grad_norm': 6.931077480316162, 'learning_rate': 4.728171553810861e-05, 'epoch': 0.16}
{'loss': 1.4956, 'grad_norm': 10.174436569213867, 'learning_rate': 4.727896423247653e-05, 'epoch': 0.16}
{'loss': 1.484, 'grad_norm': 11.743756294250488, 'learning_rate': 4.7276212926844436e-05, 'epoch': 0.16}
{'loss': 1.4673, 'grad_norm': 13.401228904724121, 'learning_rate': 4.727346162121235e-05, 'epoch': 0.16}
{'loss': 1.4923, 'grad_norm': 27.867998123168945, 'learning_rate': 4.7270710315580267e-05, 'epoch': 0.16}
{'loss': 1.476, 'grad_norm': 8.426937103271484, 'learning_rate': 4.726795900994817e-05, 'epoch': 0.16}
{'loss': 1.466, 'grad_norm': 9.039795875549316, 'learning_rate': 4.726520770431608e-05, 'epoch': 0.16}
{'loss': 1.4634, 'grad_norm': 9.012330055236816, 'learning_rate': 4.7262456398684e-05, 'epoch': 0.16}
{'loss': 1.488, 'grad_norm': 6.839291095733643, 'learning_rate': 4.725970509305191e-05, 'epoch': 0.16}
{'loss': 1.4501, 'grad_norm': 7.381160259246826, 'learning_rate': 4.725695378741982e-05, 'epoch': 0.16}
{'loss': 1.5043, 'grad_norm': 7.075179100036621, 'learning_rate': 4.725420248178774e-05, 'epoch': 0.16}
{'loss': 1.4889, 'grad_norm': 12.77310562133789, 'learning_rate': 4.7251451176155645e-05, 'epoch': 0.16}
{'loss': 1.4964, 'grad_norm': 8.807293891906738, 'learning_rate': 4.724869987052356e-05, 'epoch': 0.17}
{'loss': 1.4864, 'grad_norm': 11.664904594421387, 'learning_rate': 4.7245948564891476e-05, 'epoch': 0.17}
{'loss': 1.4895, 'grad_norm': 11.657865524291992, 'learning_rate': 4.7243197259259384e-05, 'epoch': 0.17}
{'loss': 1.4882, 'grad_norm': 9.069992065429688, 'learning_rate': 4.724044595362729e-05, 'epoch': 0.17}
{'loss': 1.4903, 'grad_norm': 11.038657188415527, 'learning_rate': 4.723769464799521e-05, 'epoch': 0.17}
{'loss': 1.4621, 'grad_norm': 9.848742485046387, 'learning_rate': 4.7234943342363116e-05, 'epoch': 0.17}
{'loss': 1.4985, 'grad_norm': 9.173456192016602, 'learning_rate': 4.723219203673103e-05, 'epoch': 0.17}
{'loss': 1.4835, 'grad_norm': 6.993440628051758, 'learning_rate': 4.7229440731098946e-05, 'epoch': 0.17}
{'loss': 1.4705, 'grad_norm': 10.926280975341797, 'learning_rate': 4.7226689425466855e-05, 'epoch': 0.17}
{'loss': 1.4854, 'grad_norm': 7.930708885192871, 'learning_rate': 4.722393811983477e-05, 'epoch': 0.17}
{'loss': 1.4857, 'grad_norm': 9.801553726196289, 'learning_rate': 4.7221186814202685e-05, 'epoch': 0.17}
{'loss': 1.4659, 'grad_norm': 8.580015182495117, 'learning_rate': 4.721843550857059e-05, 'epoch': 0.17}
{'loss': 1.4793, 'grad_norm': 9.681512832641602, 'learning_rate': 4.721568420293851e-05, 'epoch': 0.17}
{'loss': 1.5136, 'grad_norm': 11.841506958007812, 'learning_rate': 4.721293289730642e-05, 'epoch': 0.17}
{'loss': 1.4617, 'grad_norm': 9.392284393310547, 'learning_rate': 4.721018159167433e-05, 'epoch': 0.17}
{'loss': 1.4967, 'grad_norm': 10.530618667602539, 'learning_rate': 4.720743028604224e-05, 'epoch': 0.17}
{'loss': 1.4746, 'grad_norm': 8.618043899536133, 'learning_rate': 4.7204678980410155e-05, 'epoch': 0.17}
{'loss': 1.499, 'grad_norm': 7.976864337921143, 'learning_rate': 4.720192767477807e-05, 'epoch': 0.17}
{'loss': 1.4752, 'grad_norm': 9.879810333251953, 'learning_rate': 4.719917636914598e-05, 'epoch': 0.17}
{'loss': 1.4781, 'grad_norm': 11.524188041687012, 'learning_rate': 4.7196425063513894e-05, 'epoch': 0.17}
{'loss': 1.4896, 'grad_norm': 7.185775279998779, 'learning_rate': 4.719367375788181e-05, 'epoch': 0.17}
{'loss': 1.469, 'grad_norm': 7.966792106628418, 'learning_rate': 4.719092245224972e-05, 'epoch': 0.17}
{'loss': 1.4853, 'grad_norm': 9.657007217407227, 'learning_rate': 4.718817114661763e-05, 'epoch': 0.17}
{'loss': 1.4474, 'grad_norm': 8.677680015563965, 'learning_rate': 4.718541984098554e-05, 'epoch': 0.17}
{'loss': 1.4749, 'grad_norm': 7.666914463043213, 'learning_rate': 4.718266853535345e-05, 'epoch': 0.17}
{'loss': 1.4762, 'grad_norm': 14.173002243041992, 'learning_rate': 4.7179917229721365e-05, 'epoch': 0.17}
{'loss': 1.4842, 'grad_norm': 10.791522026062012, 'learning_rate': 4.717716592408928e-05, 'epoch': 0.17}
{'loss': 1.488, 'grad_norm': 7.679072380065918, 'learning_rate': 4.717441461845719e-05, 'epoch': 0.17}
{'loss': 1.4868, 'grad_norm': 9.732560157775879, 'learning_rate': 4.71716633128251e-05, 'epoch': 0.17}
{'loss': 1.4703, 'grad_norm': 10.155884742736816, 'learning_rate': 4.716891200719302e-05, 'epoch': 0.17}
{'loss': 1.4647, 'grad_norm': 5.959256649017334, 'learning_rate': 4.716616070156093e-05, 'epoch': 0.17}
{'loss': 1.4532, 'grad_norm': 11.353148460388184, 'learning_rate': 4.716340939592884e-05, 'epoch': 0.17}
{'loss': 1.4884, 'grad_norm': 10.36120319366455, 'learning_rate': 4.716065809029675e-05, 'epoch': 0.17}
{'loss': 1.4711, 'grad_norm': 4.827858924865723, 'learning_rate': 4.7157906784664666e-05, 'epoch': 0.17}
{'loss': 1.4808, 'grad_norm': 9.368616104125977, 'learning_rate': 4.7155155479032574e-05, 'epoch': 0.17}
{'loss': 1.4955, 'grad_norm': 10.763655662536621, 'learning_rate': 4.715240417340049e-05, 'epoch': 0.17}
{'loss': 1.5057, 'grad_norm': 8.108057975769043, 'learning_rate': 4.7149652867768404e-05, 'epoch': 0.17}
{'loss': 1.4808, 'grad_norm': 7.08778715133667, 'learning_rate': 4.714690156213631e-05, 'epoch': 0.17}
{'loss': 1.4785, 'grad_norm': 11.921842575073242, 'learning_rate': 4.714415025650423e-05, 'epoch': 0.17}
{'loss': 1.4684, 'grad_norm': 14.769454956054688, 'learning_rate': 4.714139895087214e-05, 'epoch': 0.17}
{'loss': 1.4701, 'grad_norm': 8.66366195678711, 'learning_rate': 4.713864764524005e-05, 'epoch': 0.17}
{'loss': 1.4945, 'grad_norm': 8.049585342407227, 'learning_rate': 4.7135896339607966e-05, 'epoch': 0.17}
{'loss': 1.4763, 'grad_norm': 9.080016136169434, 'learning_rate': 4.7133145033975875e-05, 'epoch': 0.17}
{'loss': 1.482, 'grad_norm': 6.91448974609375, 'learning_rate': 4.713039372834378e-05, 'epoch': 0.17}
{'loss': 1.4644, 'grad_norm': 10.197025299072266, 'learning_rate': 4.71276424227117e-05, 'epoch': 0.17}
{'loss': 1.4954, 'grad_norm': 10.107752799987793, 'learning_rate': 4.7124891117079613e-05, 'epoch': 0.17}
{'loss': 1.501, 'grad_norm': 9.560768127441406, 'learning_rate': 4.712213981144752e-05, 'epoch': 0.17}
{'loss': 1.4838, 'grad_norm': 13.69091796875, 'learning_rate': 4.711938850581544e-05, 'epoch': 0.17}
{'loss': 1.4691, 'grad_norm': 10.135381698608398, 'learning_rate': 4.711663720018335e-05, 'epoch': 0.17}
{'loss': 1.485, 'grad_norm': inf, 'learning_rate': 4.711388589455126e-05, 'epoch': 0.17}
{'loss': 1.4716, 'grad_norm': 12.192312240600586, 'learning_rate': 4.7111134588919176e-05, 'epoch': 0.17}
{'loss': 1.4716, 'grad_norm': 8.330598831176758, 'learning_rate': 4.710838328328709e-05, 'epoch': 0.17}
{'loss': 1.4987, 'grad_norm': 10.034501075744629, 'learning_rate': 4.7105631977655e-05, 'epoch': 0.17}
{'loss': 1.5176, 'grad_norm': 10.290061950683594, 'learning_rate': 4.710288067202291e-05, 'epoch': 0.17}
{'loss': 1.4654, 'grad_norm': 12.0889253616333, 'learning_rate': 4.710012936639082e-05, 'epoch': 0.17}
{'loss': 1.4819, 'grad_norm': 7.820587635040283, 'learning_rate': 4.709737806075874e-05, 'epoch': 0.17}
{'loss': 1.4775, 'grad_norm': 8.150120735168457, 'learning_rate': 4.7094626755126646e-05, 'epoch': 0.17}
{'loss': 1.4909, 'grad_norm': 12.647221565246582, 'learning_rate': 4.709187544949456e-05, 'epoch': 0.17}
{'loss': 1.4908, 'grad_norm': 12.30191707611084, 'learning_rate': 4.7089124143862476e-05, 'epoch': 0.17}
{'loss': 1.485, 'grad_norm': 6.438595771789551, 'learning_rate': 4.7086372838230385e-05, 'epoch': 0.17}
{'loss': 1.4859, 'grad_norm': 9.41541576385498, 'learning_rate': 4.70836215325983e-05, 'epoch': 0.17}
{'loss': 1.4721, 'grad_norm': 9.583277702331543, 'learning_rate': 4.708087022696621e-05, 'epoch': 0.18}
{'loss': 1.4596, 'grad_norm': 10.226386070251465, 'learning_rate': 4.707811892133412e-05, 'epoch': 0.18}
{'loss': 1.4703, 'grad_norm': 10.9879732131958, 'learning_rate': 4.707536761570203e-05, 'epoch': 0.18}
{'loss': 1.4899, 'grad_norm': 9.273009300231934, 'learning_rate': 4.707261631006995e-05, 'epoch': 0.18}
{'loss': 1.4842, 'grad_norm': 11.721781730651855, 'learning_rate': 4.7069865004437855e-05, 'epoch': 0.18}
{'loss': 1.499, 'grad_norm': 10.819595336914062, 'learning_rate': 4.706711369880577e-05, 'epoch': 0.18}
{'loss': 1.4787, 'grad_norm': 9.173372268676758, 'learning_rate': 4.7064362393173686e-05, 'epoch': 0.18}
{'loss': 1.4973, 'grad_norm': 10.156161308288574, 'learning_rate': 4.7061611087541594e-05, 'epoch': 0.18}
{'loss': 1.487, 'grad_norm': 11.506617546081543, 'learning_rate': 4.705885978190951e-05, 'epoch': 0.18}
{'loss': 1.4788, 'grad_norm': 7.292954444885254, 'learning_rate': 4.7056108476277424e-05, 'epoch': 0.18}
{'loss': 1.4901, 'grad_norm': 12.675764083862305, 'learning_rate': 4.705335717064533e-05, 'epoch': 0.18}
{'loss': 1.5067, 'grad_norm': 9.394784927368164, 'learning_rate': 4.705060586501324e-05, 'epoch': 0.18}
{'loss': 1.4632, 'grad_norm': 8.5784273147583, 'learning_rate': 4.7047854559381156e-05, 'epoch': 0.18}
{'loss': 1.4907, 'grad_norm': 9.689364433288574, 'learning_rate': 4.7045103253749065e-05, 'epoch': 0.18}
{'loss': 1.4692, 'grad_norm': 6.408139705657959, 'learning_rate': 4.704235194811698e-05, 'epoch': 0.18}
{'loss': 1.4672, 'grad_norm': 11.811112403869629, 'learning_rate': 4.7039600642484895e-05, 'epoch': 0.18}
{'loss': 1.475, 'grad_norm': 8.478165626525879, 'learning_rate': 4.70368493368528e-05, 'epoch': 0.18}
{'loss': 1.4584, 'grad_norm': 10.520864486694336, 'learning_rate': 4.703409803122072e-05, 'epoch': 0.18}
{'loss': 1.4652, 'grad_norm': 6.6351776123046875, 'learning_rate': 4.7031346725588633e-05, 'epoch': 0.18}
{'loss': 1.4642, 'grad_norm': 6.784965515136719, 'learning_rate': 4.702859541995654e-05, 'epoch': 0.18}
{'loss': 1.4601, 'grad_norm': 9.554871559143066, 'learning_rate': 4.702584411432446e-05, 'epoch': 0.18}
{'loss': 1.4504, 'grad_norm': 6.760406970977783, 'learning_rate': 4.7023092808692365e-05, 'epoch': 0.18}
{'loss': 1.4832, 'grad_norm': 9.797229766845703, 'learning_rate': 4.702034150306028e-05, 'epoch': 0.18}
{'loss': 1.4803, 'grad_norm': 11.899965286254883, 'learning_rate': 4.701759019742819e-05, 'epoch': 0.18}
{'loss': 1.4553, 'grad_norm': 9.460926055908203, 'learning_rate': 4.7014838891796104e-05, 'epoch': 0.18}
{'loss': 1.4895, 'grad_norm': 12.171648979187012, 'learning_rate': 4.701208758616402e-05, 'epoch': 0.18}
{'loss': 1.4791, 'grad_norm': 8.858613967895508, 'learning_rate': 4.700933628053193e-05, 'epoch': 0.18}
{'loss': 1.4736, 'grad_norm': 8.259005546569824, 'learning_rate': 4.700658497489984e-05, 'epoch': 0.18}
{'loss': 1.4715, 'grad_norm': 7.4567060470581055, 'learning_rate': 4.700383366926776e-05, 'epoch': 0.18}
{'loss': 1.4636, 'grad_norm': 8.023266792297363, 'learning_rate': 4.7001082363635666e-05, 'epoch': 0.18}
{'loss': 1.4639, 'grad_norm': 7.393029689788818, 'learning_rate': 4.6998331058003575e-05, 'epoch': 0.18}
{'loss': 1.48, 'grad_norm': 8.07656478881836, 'learning_rate': 4.699557975237149e-05, 'epoch': 0.18}
{'loss': 1.4859, 'grad_norm': 9.26103687286377, 'learning_rate': 4.69928284467394e-05, 'epoch': 0.18}
{'loss': 1.4593, 'grad_norm': 17.236347198486328, 'learning_rate': 4.699007714110731e-05, 'epoch': 0.18}
{'loss': 1.4804, 'grad_norm': 10.634861946105957, 'learning_rate': 4.698732583547523e-05, 'epoch': 0.18}
{'loss': 1.4967, 'grad_norm': 8.00877857208252, 'learning_rate': 4.698457452984314e-05, 'epoch': 0.18}
{'loss': 1.4791, 'grad_norm': 10.931817054748535, 'learning_rate': 4.698182322421105e-05, 'epoch': 0.18}
{'loss': 1.4762, 'grad_norm': 8.090311050415039, 'learning_rate': 4.697907191857897e-05, 'epoch': 0.18}
{'loss': 1.4928, 'grad_norm': 10.221521377563477, 'learning_rate': 4.6976320612946875e-05, 'epoch': 0.18}
{'loss': 1.4889, 'grad_norm': 9.890180587768555, 'learning_rate': 4.697356930731479e-05, 'epoch': 0.18}
{'loss': 1.53, 'grad_norm': 8.632936477661133, 'learning_rate': 4.69708180016827e-05, 'epoch': 0.18}
{'loss': 1.4666, 'grad_norm': 8.986140251159668, 'learning_rate': 4.6968066696050614e-05, 'epoch': 0.18}
{'loss': 1.4551, 'grad_norm': 9.966825485229492, 'learning_rate': 4.696531539041852e-05, 'epoch': 0.18}
{'loss': 1.4706, 'grad_norm': 7.881124973297119, 'learning_rate': 4.696256408478644e-05, 'epoch': 0.18}
{'loss': 1.4696, 'grad_norm': 11.763664245605469, 'learning_rate': 4.695981277915435e-05, 'epoch': 0.18}
{'loss': 1.4793, 'grad_norm': 7.786286354064941, 'learning_rate': 4.695706147352226e-05, 'epoch': 0.18}
{'loss': 1.4839, 'grad_norm': 9.363397598266602, 'learning_rate': 4.6954310167890176e-05, 'epoch': 0.18}
{'loss': 1.4783, 'grad_norm': 6.5223469734191895, 'learning_rate': 4.695155886225809e-05, 'epoch': 0.18}
{'loss': 1.5021, 'grad_norm': 7.594969749450684, 'learning_rate': 4.6948807556626e-05, 'epoch': 0.18}
{'loss': 1.4867, 'grad_norm': 8.833131790161133, 'learning_rate': 4.6946056250993915e-05, 'epoch': 0.18}
{'loss': 1.5043, 'grad_norm': 11.668055534362793, 'learning_rate': 4.694330494536182e-05, 'epoch': 0.18}
{'loss': 1.4879, 'grad_norm': 9.198160171508789, 'learning_rate': 4.694055363972973e-05, 'epoch': 0.18}
{'loss': 1.487, 'grad_norm': 11.395952224731445, 'learning_rate': 4.693780233409765e-05, 'epoch': 0.18}
{'loss': 1.518, 'grad_norm': 11.178462028503418, 'learning_rate': 4.693505102846556e-05, 'epoch': 0.18}
{'loss': 1.5018, 'grad_norm': 9.681427001953125, 'learning_rate': 4.693229972283347e-05, 'epoch': 0.18}
{'loss': 1.4779, 'grad_norm': 9.515279769897461, 'learning_rate': 4.6929548417201385e-05, 'epoch': 0.18}
{'loss': 1.4811, 'grad_norm': 7.066597938537598, 'learning_rate': 4.69267971115693e-05, 'epoch': 0.18}
{'loss': 1.5181, 'grad_norm': 9.136130332946777, 'learning_rate': 4.692404580593721e-05, 'epoch': 0.18}
{'loss': 1.5306, 'grad_norm': 10.587213516235352, 'learning_rate': 4.6921294500305124e-05, 'epoch': 0.18}
{'loss': 1.5076, 'grad_norm': 8.466611862182617, 'learning_rate': 4.691854319467304e-05, 'epoch': 0.18}
{'loss': 1.5411, 'grad_norm': 11.098169326782227, 'learning_rate': 4.691579188904095e-05, 'epoch': 0.19}
{'loss': 1.5042, 'grad_norm': 9.542279243469238, 'learning_rate': 4.6913040583408856e-05, 'epoch': 0.19}
{'loss': 1.5021, 'grad_norm': 7.32064151763916, 'learning_rate': 4.691028927777677e-05, 'epoch': 0.19}
{'loss': 1.5042, 'grad_norm': 11.357136726379395, 'learning_rate': 4.690753797214468e-05, 'epoch': 0.19}
{'loss': 1.4923, 'grad_norm': 15.599925994873047, 'learning_rate': 4.6904786666512595e-05, 'epoch': 0.19}
{'loss': 1.4941, 'grad_norm': 10.199986457824707, 'learning_rate': 4.690203536088051e-05, 'epoch': 0.19}
{'loss': 1.5081, 'grad_norm': 11.672343254089355, 'learning_rate': 4.689928405524842e-05, 'epoch': 0.19}
{'loss': 1.5333, 'grad_norm': 8.436853408813477, 'learning_rate': 4.689653274961633e-05, 'epoch': 0.19}
{'loss': 1.5135, 'grad_norm': 11.152113914489746, 'learning_rate': 4.689378144398425e-05, 'epoch': 0.19}
{'loss': 1.5298, 'grad_norm': 8.186738014221191, 'learning_rate': 4.689103013835216e-05, 'epoch': 0.19}
{'loss': 1.9076, 'grad_norm': 93.35053253173828, 'learning_rate': 4.6888278832720065e-05, 'epoch': 0.19}
{'loss': 1.8955, 'grad_norm': 10.346670150756836, 'learning_rate': 4.688552752708798e-05, 'epoch': 0.19}
{'loss': 1.7773, 'grad_norm': 40.081153869628906, 'learning_rate': 4.6882776221455895e-05, 'epoch': 0.19}
{'loss': 1.9108, 'grad_norm': 8.335807800292969, 'learning_rate': 4.6880024915823804e-05, 'epoch': 0.19}
{'loss': 1.6543, 'grad_norm': 13.320844650268555, 'learning_rate': 4.687727361019172e-05, 'epoch': 0.19}
{'loss': 1.6799, 'grad_norm': 9.502729415893555, 'learning_rate': 4.6874522304559634e-05, 'epoch': 0.19}
{'loss': 1.7083, 'grad_norm': 12.644293785095215, 'learning_rate': 4.687177099892754e-05, 'epoch': 0.19}
{'loss': 1.7456, 'grad_norm': 9.0537748336792, 'learning_rate': 4.686901969329546e-05, 'epoch': 0.19}
{'loss': 1.8664, 'grad_norm': 192.17393493652344, 'learning_rate': 4.686626838766337e-05, 'epoch': 0.19}
{'loss': 1.6837, 'grad_norm': 8.501562118530273, 'learning_rate': 4.6863517082031274e-05, 'epoch': 0.19}
{'loss': 1.6652, 'grad_norm': 11.648345947265625, 'learning_rate': 4.686076577639919e-05, 'epoch': 0.19}
{'loss': 1.862, 'grad_norm': 8.574299812316895, 'learning_rate': 4.6858014470767105e-05, 'epoch': 0.19}
{'loss': 1.6497, 'grad_norm': 14.104843139648438, 'learning_rate': 4.685526316513501e-05, 'epoch': 0.19}
{'loss': 1.6712, 'grad_norm': 9.884845733642578, 'learning_rate': 4.685251185950293e-05, 'epoch': 0.19}
{'loss': 1.7077, 'grad_norm': 10.854574203491211, 'learning_rate': 4.684976055387084e-05, 'epoch': 0.19}
{'loss': 1.5445, 'grad_norm': 11.276516914367676, 'learning_rate': 4.684700924823875e-05, 'epoch': 0.19}
{'loss': 1.5192, 'grad_norm': 8.07763671875, 'learning_rate': 4.684425794260667e-05, 'epoch': 0.19}
{'loss': 1.4786, 'grad_norm': 7.137641906738281, 'learning_rate': 4.684150663697458e-05, 'epoch': 0.19}
{'loss': 1.566, 'grad_norm': 8.415364265441895, 'learning_rate': 4.683875533134249e-05, 'epoch': 0.19}
{'loss': 1.5324, 'grad_norm': 9.531322479248047, 'learning_rate': 4.68360040257104e-05, 'epoch': 0.19}
{'loss': 1.5375, 'grad_norm': 8.40423583984375, 'learning_rate': 4.6833252720078314e-05, 'epoch': 0.19}
{'loss': 1.5435, 'grad_norm': 10.160860061645508, 'learning_rate': 4.683050141444623e-05, 'epoch': 0.19}
{'loss': 1.5152, 'grad_norm': 6.032065391540527, 'learning_rate': 4.682775010881414e-05, 'epoch': 0.19}
{'loss': 1.4874, 'grad_norm': 10.506930351257324, 'learning_rate': 4.682499880318205e-05, 'epoch': 0.19}
{'loss': 1.4968, 'grad_norm': 8.855704307556152, 'learning_rate': 4.682224749754997e-05, 'epoch': 0.19}
{'loss': 1.5144, 'grad_norm': 10.082674026489258, 'learning_rate': 4.6819496191917876e-05, 'epoch': 0.19}
{'loss': 1.4852, 'grad_norm': 12.586864471435547, 'learning_rate': 4.681674488628579e-05, 'epoch': 0.19}
{'loss': 1.4763, 'grad_norm': 7.962011337280273, 'learning_rate': 4.6813993580653706e-05, 'epoch': 0.19}
{'loss': 1.5224, 'grad_norm': 12.36872673034668, 'learning_rate': 4.6811242275021615e-05, 'epoch': 0.19}
{'loss': 1.4949, 'grad_norm': 19.2633113861084, 'learning_rate': 4.680849096938952e-05, 'epoch': 0.19}
{'loss': 1.5625, 'grad_norm': 8.47352123260498, 'learning_rate': 4.680573966375744e-05, 'epoch': 0.19}
{'loss': 1.4659, 'grad_norm': 8.851982116699219, 'learning_rate': 4.6802988358125347e-05, 'epoch': 0.19}
{'loss': 1.4729, 'grad_norm': 10.110154151916504, 'learning_rate': 4.680023705249326e-05, 'epoch': 0.19}
{'loss': 1.4697, 'grad_norm': 10.0674467086792, 'learning_rate': 4.679748574686118e-05, 'epoch': 0.19}
{'loss': 1.5067, 'grad_norm': 9.866599082946777, 'learning_rate': 4.6794734441229085e-05, 'epoch': 0.19}
{'loss': 1.4634, 'grad_norm': 8.901447296142578, 'learning_rate': 4.6791983135597e-05, 'epoch': 0.19}
{'loss': 1.5622, 'grad_norm': 8.106250762939453, 'learning_rate': 4.6789231829964916e-05, 'epoch': 0.19}
{'loss': 1.528, 'grad_norm': 12.805727005004883, 'learning_rate': 4.6786480524332824e-05, 'epoch': 0.19}
{'loss': 1.5656, 'grad_norm': 6.691972732543945, 'learning_rate': 4.678372921870074e-05, 'epoch': 0.19}
{'loss': 1.5801, 'grad_norm': 8.517285346984863, 'learning_rate': 4.678097791306865e-05, 'epoch': 0.19}
{'loss': 1.6256, 'grad_norm': 11.097503662109375, 'learning_rate': 4.677822660743656e-05, 'epoch': 0.19}
{'loss': 1.546, 'grad_norm': 9.937793731689453, 'learning_rate': 4.677547530180447e-05, 'epoch': 0.19}
{'loss': 1.5192, 'grad_norm': 12.662639617919922, 'learning_rate': 4.6772723996172386e-05, 'epoch': 0.19}
{'loss': 1.5439, 'grad_norm': 9.136960983276367, 'learning_rate': 4.67699726905403e-05, 'epoch': 0.19}
{'loss': 1.5495, 'grad_norm': 8.76932430267334, 'learning_rate': 4.676722138490821e-05, 'epoch': 0.19}
{'loss': 1.5815, 'grad_norm': 7.700368881225586, 'learning_rate': 4.6764470079276125e-05, 'epoch': 0.19}
{'loss': 1.4917, 'grad_norm': 46.947364807128906, 'learning_rate': 4.676171877364404e-05, 'epoch': 0.19}
{'loss': 1.5453, 'grad_norm': 11.348325729370117, 'learning_rate': 4.675896746801195e-05, 'epoch': 0.19}
{'loss': 1.5022, 'grad_norm': 9.228842735290527, 'learning_rate': 4.6756216162379863e-05, 'epoch': 0.19}
{'loss': 1.5007, 'grad_norm': 9.173401832580566, 'learning_rate': 4.675346485674777e-05, 'epoch': 0.19}
{'loss': 1.4727, 'grad_norm': 8.030966758728027, 'learning_rate': 4.675071355111568e-05, 'epoch': 0.19}
{'loss': 1.5039, 'grad_norm': 11.022141456604004, 'learning_rate': 4.6747962245483595e-05, 'epoch': 0.2}
{'loss': 1.4892, 'grad_norm': 8.06417179107666, 'learning_rate': 4.674521093985151e-05, 'epoch': 0.2}
{'loss': 1.5393, 'grad_norm': 7.556422233581543, 'learning_rate': 4.674245963421942e-05, 'epoch': 0.2}
{'loss': 1.4908, 'grad_norm': 8.32551097869873, 'learning_rate': 4.6739708328587334e-05, 'epoch': 0.2}
{'loss': 1.4788, 'grad_norm': 10.179616928100586, 'learning_rate': 4.673695702295525e-05, 'epoch': 0.2}
{'loss': 1.5584, 'grad_norm': 12.20495319366455, 'learning_rate': 4.673420571732316e-05, 'epoch': 0.2}
{'loss': 1.7769, 'grad_norm': 7.421200752258301, 'learning_rate': 4.673145441169107e-05, 'epoch': 0.2}
{'loss': 1.6271, 'grad_norm': 11.927038192749023, 'learning_rate': 4.672870310605899e-05, 'epoch': 0.2}
{'loss': 1.6474, 'grad_norm': 17.31012535095215, 'learning_rate': 4.672595180042689e-05, 'epoch': 0.2}
{'loss': 1.5029, 'grad_norm': 10.42601203918457, 'learning_rate': 4.6723200494794805e-05, 'epoch': 0.2}
{'loss': 1.5191, 'grad_norm': 12.56181812286377, 'learning_rate': 4.672044918916272e-05, 'epoch': 0.2}
{'loss': 1.5535, 'grad_norm': 10.608118057250977, 'learning_rate': 4.671769788353063e-05, 'epoch': 0.2}
{'loss': 1.4849, 'grad_norm': 9.677628517150879, 'learning_rate': 4.671494657789854e-05, 'epoch': 0.2}
{'loss': 1.4775, 'grad_norm': 11.383793830871582, 'learning_rate': 4.671219527226646e-05, 'epoch': 0.2}
{'loss': 1.4922, 'grad_norm': 6.574410438537598, 'learning_rate': 4.670944396663437e-05, 'epoch': 0.2}
{'loss': 1.4967, 'grad_norm': 6.476960182189941, 'learning_rate': 4.670669266100228e-05, 'epoch': 0.2}
{'loss': 1.4754, 'grad_norm': 21.84545135498047, 'learning_rate': 4.67039413553702e-05, 'epoch': 0.2}
{'loss': 1.4947, 'grad_norm': 51.138301849365234, 'learning_rate': 4.6701190049738105e-05, 'epoch': 0.2}
{'loss': 1.5092, 'grad_norm': 11.48706340789795, 'learning_rate': 4.6698438744106014e-05, 'epoch': 0.2}
{'loss': 1.4539, 'grad_norm': 11.139127731323242, 'learning_rate': 4.669568743847393e-05, 'epoch': 0.2}
{'loss': 1.4896, 'grad_norm': 9.863777160644531, 'learning_rate': 4.6692936132841844e-05, 'epoch': 0.2}
{'loss': 1.4993, 'grad_norm': 5.85535192489624, 'learning_rate': 4.669018482720975e-05, 'epoch': 0.2}
{'loss': 1.4774, 'grad_norm': 6.875484466552734, 'learning_rate': 4.668743352157767e-05, 'epoch': 0.2}
{'loss': 1.5068, 'grad_norm': 8.826404571533203, 'learning_rate': 4.668468221594558e-05, 'epoch': 0.2}
{'loss': 1.5018, 'grad_norm': 25.342987060546875, 'learning_rate': 4.668193091031349e-05, 'epoch': 0.2}
{'loss': 1.4647, 'grad_norm': 10.56624984741211, 'learning_rate': 4.6679179604681406e-05, 'epoch': 0.2}
{'loss': 1.5389, 'grad_norm': 11.435686111450195, 'learning_rate': 4.667642829904932e-05, 'epoch': 0.2}
{'loss': 1.5091, 'grad_norm': 8.88365650177002, 'learning_rate': 4.667367699341722e-05, 'epoch': 0.2}
{'loss': 1.4885, 'grad_norm': 13.210683822631836, 'learning_rate': 4.667092568778514e-05, 'epoch': 0.2}
{'loss': 1.4779, 'grad_norm': 10.250344276428223, 'learning_rate': 4.666817438215305e-05, 'epoch': 0.2}
{'loss': 1.4792, 'grad_norm': 10.49502944946289, 'learning_rate': 4.666542307652096e-05, 'epoch': 0.2}
{'loss': 1.4862, 'grad_norm': 8.006745338439941, 'learning_rate': 4.666267177088888e-05, 'epoch': 0.2}
{'loss': 1.5362, 'grad_norm': 12.33854866027832, 'learning_rate': 4.665992046525679e-05, 'epoch': 0.2}
{'loss': 1.4911, 'grad_norm': 7.550539016723633, 'learning_rate': 4.66571691596247e-05, 'epoch': 0.2}
{'loss': 1.4813, 'grad_norm': 9.693655014038086, 'learning_rate': 4.6654417853992615e-05, 'epoch': 0.2}
{'loss': 1.5151, 'grad_norm': 9.536276817321777, 'learning_rate': 4.665166654836053e-05, 'epoch': 0.2}
{'loss': 1.4702, 'grad_norm': 10.375001907348633, 'learning_rate': 4.664891524272844e-05, 'epoch': 0.2}
{'loss': 1.4821, 'grad_norm': 7.297208309173584, 'learning_rate': 4.664616393709635e-05, 'epoch': 0.2}
{'loss': 1.4725, 'grad_norm': 11.78661823272705, 'learning_rate': 4.664341263146426e-05, 'epoch': 0.2}
{'loss': 1.4885, 'grad_norm': 8.633804321289062, 'learning_rate': 4.664066132583218e-05, 'epoch': 0.2}
{'loss': 1.5008, 'grad_norm': 10.768777847290039, 'learning_rate': 4.6637910020200086e-05, 'epoch': 0.2}
{'loss': 1.5125, 'grad_norm': 10.527606010437012, 'learning_rate': 4.6635158714568e-05, 'epoch': 0.2}
{'loss': 1.5313, 'grad_norm': 7.249996185302734, 'learning_rate': 4.6632407408935916e-05, 'epoch': 0.2}
{'loss': 1.5132, 'grad_norm': 12.20798397064209, 'learning_rate': 4.6629656103303825e-05, 'epoch': 0.2}
{'loss': 1.5028, 'grad_norm': 9.748438835144043, 'learning_rate': 4.662690479767174e-05, 'epoch': 0.2}
{'loss': 1.4627, 'grad_norm': 11.743308067321777, 'learning_rate': 4.6624153492039655e-05, 'epoch': 0.2}
{'loss': 1.4882, 'grad_norm': 5.335226535797119, 'learning_rate': 4.662140218640756e-05, 'epoch': 0.2}
{'loss': 1.5034, 'grad_norm': 10.862408638000488, 'learning_rate': 4.661865088077547e-05, 'epoch': 0.2}
{'loss': 1.5033, 'grad_norm': 11.236479759216309, 'learning_rate': 4.661589957514339e-05, 'epoch': 0.2}
{'loss': 1.4635, 'grad_norm': 10.117016792297363, 'learning_rate': 4.6613148269511295e-05, 'epoch': 0.2}
{'loss': 1.527, 'grad_norm': 9.85024356842041, 'learning_rate': 4.661039696387921e-05, 'epoch': 0.2}
{'loss': 1.4805, 'grad_norm': 13.97513484954834, 'learning_rate': 4.6607645658247125e-05, 'epoch': 0.2}
{'loss': 1.4973, 'grad_norm': 8.44267463684082, 'learning_rate': 4.6604894352615034e-05, 'epoch': 0.2}
{'loss': 1.4923, 'grad_norm': 9.009334564208984, 'learning_rate': 4.660214304698295e-05, 'epoch': 0.2}
{'loss': 1.4731, 'grad_norm': 6.029156684875488, 'learning_rate': 4.6599391741350864e-05, 'epoch': 0.2}
{'loss': 1.4995, 'grad_norm': 9.995012283325195, 'learning_rate': 4.659664043571877e-05, 'epoch': 0.2}
{'loss': 1.4675, 'grad_norm': 8.99931812286377, 'learning_rate': 4.659388913008669e-05, 'epoch': 0.2}
{'loss': 1.4417, 'grad_norm': 14.868406295776367, 'learning_rate': 4.6591137824454596e-05, 'epoch': 0.2}
{'loss': 1.5127, 'grad_norm': 122.35354614257812, 'learning_rate': 4.6588386518822504e-05, 'epoch': 0.2}
{'loss': 1.5306, 'grad_norm': 7.160025596618652, 'learning_rate': 4.658563521319042e-05, 'epoch': 0.2}
{'loss': 1.5244, 'grad_norm': 9.472847938537598, 'learning_rate': 4.6582883907558335e-05, 'epoch': 0.21}
{'loss': 1.5013, 'grad_norm': 5.863465785980225, 'learning_rate': 4.658013260192624e-05, 'epoch': 0.21}
{'loss': 1.4813, 'grad_norm': 10.023730278015137, 'learning_rate': 4.657738129629416e-05, 'epoch': 0.21}
{'loss': 1.4905, 'grad_norm': 8.052207946777344, 'learning_rate': 4.657462999066207e-05, 'epoch': 0.21}
{'loss': 1.5143, 'grad_norm': 8.764335632324219, 'learning_rate': 4.657187868502998e-05, 'epoch': 0.21}
{'loss': 1.4435, 'grad_norm': 9.455279350280762, 'learning_rate': 4.65691273793979e-05, 'epoch': 0.21}
{'loss': 1.4669, 'grad_norm': 7.035670280456543, 'learning_rate': 4.6566376073765805e-05, 'epoch': 0.21}
{'loss': 1.4746, 'grad_norm': 8.831449508666992, 'learning_rate': 4.656362476813372e-05, 'epoch': 0.21}
{'loss': 1.4951, 'grad_norm': 8.050838470458984, 'learning_rate': 4.656087346250163e-05, 'epoch': 0.21}
{'loss': 1.5062, 'grad_norm': 9.838800430297852, 'learning_rate': 4.6558122156869544e-05, 'epoch': 0.21}
{'loss': 1.4938, 'grad_norm': 10.127032279968262, 'learning_rate': 4.655537085123746e-05, 'epoch': 0.21}
{'loss': 1.4679, 'grad_norm': 14.958271980285645, 'learning_rate': 4.655261954560537e-05, 'epoch': 0.21}
{'loss': 1.4939, 'grad_norm': 8.895020484924316, 'learning_rate': 4.654986823997328e-05, 'epoch': 0.21}
{'loss': 1.4916, 'grad_norm': 8.870376586914062, 'learning_rate': 4.65471169343412e-05, 'epoch': 0.21}
{'loss': 1.5054, 'grad_norm': 10.156817436218262, 'learning_rate': 4.6544365628709106e-05, 'epoch': 0.21}
{'loss': 1.4903, 'grad_norm': 7.338503360748291, 'learning_rate': 4.654161432307702e-05, 'epoch': 0.21}
{'loss': 1.4827, 'grad_norm': 10.142105102539062, 'learning_rate': 4.653886301744493e-05, 'epoch': 0.21}
{'loss': 1.4637, 'grad_norm': 8.157305717468262, 'learning_rate': 4.653611171181284e-05, 'epoch': 0.21}
{'loss': 1.5269, 'grad_norm': 8.939006805419922, 'learning_rate': 4.653336040618075e-05, 'epoch': 0.21}
{'loss': 1.5061, 'grad_norm': 6.322612285614014, 'learning_rate': 4.653060910054867e-05, 'epoch': 0.21}
{'loss': 1.4899, 'grad_norm': 5.562066555023193, 'learning_rate': 4.6527857794916577e-05, 'epoch': 0.21}
{'loss': 1.4808, 'grad_norm': 8.341944694519043, 'learning_rate': 4.652510648928449e-05, 'epoch': 0.21}
{'loss': 1.4783, 'grad_norm': 8.12209701538086, 'learning_rate': 4.652235518365241e-05, 'epoch': 0.21}
{'loss': 1.4899, 'grad_norm': 9.313494682312012, 'learning_rate': 4.6519603878020315e-05, 'epoch': 0.21}
{'loss': 1.4542, 'grad_norm': 9.043065071105957, 'learning_rate': 4.651685257238823e-05, 'epoch': 0.21}
{'loss': 1.4886, 'grad_norm': 8.712642669677734, 'learning_rate': 4.6514101266756146e-05, 'epoch': 0.21}
{'loss': 1.4733, 'grad_norm': 9.540613174438477, 'learning_rate': 4.6511349961124054e-05, 'epoch': 0.21}
{'loss': 1.4868, 'grad_norm': 9.108193397521973, 'learning_rate': 4.650859865549196e-05, 'epoch': 0.21}
{'loss': 1.4904, 'grad_norm': 8.8296537399292, 'learning_rate': 4.650584734985988e-05, 'epoch': 0.21}
{'loss': 1.5213, 'grad_norm': 10.029557228088379, 'learning_rate': 4.650309604422779e-05, 'epoch': 0.21}
{'loss': 1.5025, 'grad_norm': 9.390090942382812, 'learning_rate': 4.65003447385957e-05, 'epoch': 0.21}
{'loss': 1.4637, 'grad_norm': 7.117425918579102, 'learning_rate': 4.6497593432963616e-05, 'epoch': 0.21}
{'loss': 1.4751, 'grad_norm': 8.363322257995605, 'learning_rate': 4.649484212733153e-05, 'epoch': 0.21}
{'loss': 1.4599, 'grad_norm': 10.302263259887695, 'learning_rate': 4.649209082169944e-05, 'epoch': 0.21}
{'loss': 1.4986, 'grad_norm': 9.331538200378418, 'learning_rate': 4.6489339516067355e-05, 'epoch': 0.21}
{'loss': 1.5, 'grad_norm': 9.322526931762695, 'learning_rate': 4.648658821043527e-05, 'epoch': 0.21}
{'loss': 1.4982, 'grad_norm': 10.244661331176758, 'learning_rate': 4.648383690480317e-05, 'epoch': 0.21}
{'loss': 1.5009, 'grad_norm': 10.29520320892334, 'learning_rate': 4.6481085599171087e-05, 'epoch': 0.21}
{'loss': 1.4845, 'grad_norm': 7.543183326721191, 'learning_rate': 4.6478334293539e-05, 'epoch': 0.21}
{'loss': 1.463, 'grad_norm': 7.254530429840088, 'learning_rate': 4.647558298790691e-05, 'epoch': 0.21}
{'loss': 1.4948, 'grad_norm': 11.944664001464844, 'learning_rate': 4.6472831682274825e-05, 'epoch': 0.21}
{'loss': 1.4939, 'grad_norm': 7.752316951751709, 'learning_rate': 4.647008037664274e-05, 'epoch': 0.21}
{'loss': 1.4782, 'grad_norm': 8.744604110717773, 'learning_rate': 4.646732907101065e-05, 'epoch': 0.21}
{'loss': 1.4904, 'grad_norm': 9.648715019226074, 'learning_rate': 4.6464577765378564e-05, 'epoch': 0.21}
{'loss': 1.4479, 'grad_norm': 6.574150085449219, 'learning_rate': 4.646182645974648e-05, 'epoch': 0.21}
{'loss': 1.4715, 'grad_norm': 8.33734130859375, 'learning_rate': 4.645907515411439e-05, 'epoch': 0.21}
{'loss': 1.4609, 'grad_norm': 11.586155891418457, 'learning_rate': 4.6456323848482296e-05, 'epoch': 0.21}
{'loss': 1.4653, 'grad_norm': 7.397571563720703, 'learning_rate': 4.645357254285021e-05, 'epoch': 0.21}
{'loss': 1.4596, 'grad_norm': 10.615427017211914, 'learning_rate': 4.6450821237218126e-05, 'epoch': 0.21}
{'loss': 1.4696, 'grad_norm': 9.06356143951416, 'learning_rate': 4.6448069931586034e-05, 'epoch': 0.21}
{'loss': 1.4722, 'grad_norm': 9.998895645141602, 'learning_rate': 4.644531862595395e-05, 'epoch': 0.21}
{'loss': 1.477, 'grad_norm': 7.664145469665527, 'learning_rate': 4.6442567320321865e-05, 'epoch': 0.21}
{'loss': 1.4636, 'grad_norm': 9.377935409545898, 'learning_rate': 4.643981601468977e-05, 'epoch': 0.21}
{'loss': 1.487, 'grad_norm': 8.7444429397583, 'learning_rate': 4.643706470905769e-05, 'epoch': 0.21}
{'loss': 1.4732, 'grad_norm': 10.724617004394531, 'learning_rate': 4.64343134034256e-05, 'epoch': 0.21}
{'loss': 1.4616, 'grad_norm': 7.904392242431641, 'learning_rate': 4.643156209779351e-05, 'epoch': 0.21}
{'loss': 1.4898, 'grad_norm': 9.215457916259766, 'learning_rate': 4.642881079216142e-05, 'epoch': 0.21}
{'loss': 1.4892, 'grad_norm': 10.040008544921875, 'learning_rate': 4.6426059486529335e-05, 'epoch': 0.21}
{'loss': 1.4847, 'grad_norm': 11.141857147216797, 'learning_rate': 4.6423308180897244e-05, 'epoch': 0.21}
{'loss': 1.476, 'grad_norm': 8.10831069946289, 'learning_rate': 4.642055687526516e-05, 'epoch': 0.21}
{'loss': 1.5314, 'grad_norm': 7.447622776031494, 'learning_rate': 4.6417805569633074e-05, 'epoch': 0.21}
{'loss': 1.5091, 'grad_norm': 9.051900863647461, 'learning_rate': 4.641505426400098e-05, 'epoch': 0.22}
{'loss': 1.5322, 'grad_norm': 13.149503707885742, 'learning_rate': 4.64123029583689e-05, 'epoch': 0.22}
{'loss': 1.4824, 'grad_norm': 12.60035514831543, 'learning_rate': 4.640955165273681e-05, 'epoch': 0.22}
{'loss': 1.4694, 'grad_norm': 7.625150680541992, 'learning_rate': 4.640680034710472e-05, 'epoch': 0.22}
{'loss': 1.4676, 'grad_norm': 11.66402816772461, 'learning_rate': 4.640404904147263e-05, 'epoch': 0.22}
{'loss': 1.4851, 'grad_norm': 10.459420204162598, 'learning_rate': 4.6401297735840545e-05, 'epoch': 0.22}
{'loss': 1.4729, 'grad_norm': 7.236877918243408, 'learning_rate': 4.639854643020845e-05, 'epoch': 0.22}
{'loss': 1.5116, 'grad_norm': 9.437895774841309, 'learning_rate': 4.639579512457637e-05, 'epoch': 0.22}
{'loss': 1.4709, 'grad_norm': 8.532658576965332, 'learning_rate': 4.639304381894428e-05, 'epoch': 0.22}
{'loss': 1.4961, 'grad_norm': 6.428638935089111, 'learning_rate': 4.639029251331219e-05, 'epoch': 0.22}
{'loss': 1.4806, 'grad_norm': 13.005111694335938, 'learning_rate': 4.638754120768011e-05, 'epoch': 0.22}
{'loss': 1.4633, 'grad_norm': 8.010013580322266, 'learning_rate': 4.638478990204802e-05, 'epoch': 0.22}
{'loss': 1.4792, 'grad_norm': 12.167726516723633, 'learning_rate': 4.638203859641593e-05, 'epoch': 0.22}
{'loss': 1.465, 'grad_norm': 11.41462516784668, 'learning_rate': 4.6379287290783845e-05, 'epoch': 0.22}
{'loss': 1.4701, 'grad_norm': 10.661958694458008, 'learning_rate': 4.6376535985151754e-05, 'epoch': 0.22}
{'loss': 1.4821, 'grad_norm': 11.021636009216309, 'learning_rate': 4.637378467951967e-05, 'epoch': 0.22}
{'loss': 1.4838, 'grad_norm': 10.259963989257812, 'learning_rate': 4.637103337388758e-05, 'epoch': 0.22}
{'loss': 1.4825, 'grad_norm': 11.887959480285645, 'learning_rate': 4.636828206825549e-05, 'epoch': 0.22}
{'loss': 1.4761, 'grad_norm': 11.39128303527832, 'learning_rate': 4.636553076262341e-05, 'epoch': 0.22}
{'loss': 1.5032, 'grad_norm': 6.628098964691162, 'learning_rate': 4.6362779456991316e-05, 'epoch': 0.22}
{'loss': 1.4902, 'grad_norm': 8.262263298034668, 'learning_rate': 4.636002815135923e-05, 'epoch': 0.22}
{'loss': 1.508, 'grad_norm': 5.7857666015625, 'learning_rate': 4.6357276845727146e-05, 'epoch': 0.22}
{'loss': 1.5461, 'grad_norm': 7.572531700134277, 'learning_rate': 4.6354525540095055e-05, 'epoch': 0.22}
{'loss': 1.4808, 'grad_norm': 11.629980087280273, 'learning_rate': 4.635177423446297e-05, 'epoch': 0.22}
{'loss': 1.4647, 'grad_norm': 7.554471969604492, 'learning_rate': 4.634902292883088e-05, 'epoch': 0.22}
{'loss': 1.4529, 'grad_norm': 10.249889373779297, 'learning_rate': 4.6346271623198786e-05, 'epoch': 0.22}
{'loss': 1.4975, 'grad_norm': 8.791023254394531, 'learning_rate': 4.63435203175667e-05, 'epoch': 0.22}
{'loss': 1.4882, 'grad_norm': 7.121941089630127, 'learning_rate': 4.634076901193462e-05, 'epoch': 0.22}
{'loss': 1.4973, 'grad_norm': 9.473983764648438, 'learning_rate': 4.6338017706302525e-05, 'epoch': 0.22}
{'loss': 1.5224, 'grad_norm': 10.169795036315918, 'learning_rate': 4.633526640067044e-05, 'epoch': 0.22}
{'loss': 1.5211, 'grad_norm': 8.621929168701172, 'learning_rate': 4.6332515095038355e-05, 'epoch': 0.22}
{'loss': 1.5502, 'grad_norm': 7.361867427825928, 'learning_rate': 4.6329763789406264e-05, 'epoch': 0.22}
{'loss': 1.4592, 'grad_norm': 9.78110408782959, 'learning_rate': 4.632701248377418e-05, 'epoch': 0.22}
{'loss': 1.4682, 'grad_norm': 7.897862434387207, 'learning_rate': 4.6324261178142094e-05, 'epoch': 0.22}
{'loss': 1.4677, 'grad_norm': 8.572824478149414, 'learning_rate': 4.632150987251e-05, 'epoch': 0.22}
{'loss': 1.4675, 'grad_norm': 9.870251655578613, 'learning_rate': 4.631875856687791e-05, 'epoch': 0.22}
{'loss': 1.4456, 'grad_norm': 8.607908248901367, 'learning_rate': 4.6316007261245826e-05, 'epoch': 0.22}
{'loss': 1.4696, 'grad_norm': 9.137561798095703, 'learning_rate': 4.631325595561374e-05, 'epoch': 0.22}
{'loss': 1.4684, 'grad_norm': 11.375115394592285, 'learning_rate': 4.631050464998165e-05, 'epoch': 0.22}
{'loss': 1.482, 'grad_norm': 7.724959850311279, 'learning_rate': 4.6307753344349565e-05, 'epoch': 0.22}
{'loss': 1.4543, 'grad_norm': 9.329253196716309, 'learning_rate': 4.630500203871748e-05, 'epoch': 0.22}
{'loss': 1.454, 'grad_norm': 10.029367446899414, 'learning_rate': 4.630225073308539e-05, 'epoch': 0.22}
{'loss': 1.509, 'grad_norm': 6.999165058135986, 'learning_rate': 4.62994994274533e-05, 'epoch': 0.22}
{'loss': 1.4691, 'grad_norm': 11.942953109741211, 'learning_rate': 4.629674812182122e-05, 'epoch': 0.22}
{'loss': 1.4662, 'grad_norm': 8.789701461791992, 'learning_rate': 4.629399681618912e-05, 'epoch': 0.22}
{'loss': 1.4774, 'grad_norm': 9.00161361694336, 'learning_rate': 4.6291245510557035e-05, 'epoch': 0.22}
{'loss': 1.4512, 'grad_norm': 6.3880228996276855, 'learning_rate': 4.628849420492495e-05, 'epoch': 0.22}
{'loss': 1.4636, 'grad_norm': 8.105249404907227, 'learning_rate': 4.628574289929286e-05, 'epoch': 0.22}
{'loss': 1.4831, 'grad_norm': 8.385421752929688, 'learning_rate': 4.6282991593660774e-05, 'epoch': 0.22}
{'loss': 1.4957, 'grad_norm': 8.495889663696289, 'learning_rate': 4.628024028802869e-05, 'epoch': 0.22}
{'loss': 1.4986, 'grad_norm': 7.787834644317627, 'learning_rate': 4.62774889823966e-05, 'epoch': 0.22}
{'loss': 1.4828, 'grad_norm': 7.487082481384277, 'learning_rate': 4.627473767676451e-05, 'epoch': 0.22}
{'loss': 1.4828, 'grad_norm': 8.544161796569824, 'learning_rate': 4.627198637113243e-05, 'epoch': 0.22}
{'loss': 1.4867, 'grad_norm': 8.719870567321777, 'learning_rate': 4.6269235065500336e-05, 'epoch': 0.22}
{'loss': 1.4821, 'grad_norm': 8.463547706604004, 'learning_rate': 4.6266483759868244e-05, 'epoch': 0.22}
{'loss': 1.455, 'grad_norm': 13.30632209777832, 'learning_rate': 4.626373245423616e-05, 'epoch': 0.22}
{'loss': 1.4658, 'grad_norm': 9.16640567779541, 'learning_rate': 4.626098114860407e-05, 'epoch': 0.22}
{'loss': 1.4773, 'grad_norm': 10.534026145935059, 'learning_rate': 4.625822984297198e-05, 'epoch': 0.22}
{'loss': 1.4662, 'grad_norm': 10.130373001098633, 'learning_rate': 4.62554785373399e-05, 'epoch': 0.22}
{'loss': 1.4553, 'grad_norm': 6.849843978881836, 'learning_rate': 4.6252727231707807e-05, 'epoch': 0.22}
{'loss': 1.5194, 'grad_norm': 10.49138069152832, 'learning_rate': 4.624997592607572e-05, 'epoch': 0.23}
{'loss': 1.4985, 'grad_norm': 9.515944480895996, 'learning_rate': 4.624722462044364e-05, 'epoch': 0.23}
{'loss': 1.5206, 'grad_norm': 6.127353668212891, 'learning_rate': 4.6244473314811545e-05, 'epoch': 0.23}
{'loss': 1.4597, 'grad_norm': 9.24634838104248, 'learning_rate': 4.6241722009179454e-05, 'epoch': 0.23}
{'loss': 1.4747, 'grad_norm': 8.973910331726074, 'learning_rate': 4.623897070354737e-05, 'epoch': 0.23}
{'loss': 1.4737, 'grad_norm': 8.043646812438965, 'learning_rate': 4.6236219397915284e-05, 'epoch': 0.23}
{'loss': 1.4563, 'grad_norm': 9.818717002868652, 'learning_rate': 4.623346809228319e-05, 'epoch': 0.23}
{'loss': 1.4594, 'grad_norm': 9.994940757751465, 'learning_rate': 4.623071678665111e-05, 'epoch': 0.23}
{'loss': 1.4804, 'grad_norm': 7.8014373779296875, 'learning_rate': 4.622796548101902e-05, 'epoch': 0.23}
{'loss': 1.482, 'grad_norm': 10.560070991516113, 'learning_rate': 4.622521417538693e-05, 'epoch': 0.23}
{'loss': 1.4526, 'grad_norm': 7.225388050079346, 'learning_rate': 4.6222462869754846e-05, 'epoch': 0.23}
{'loss': 1.4856, 'grad_norm': 8.416998863220215, 'learning_rate': 4.621971156412276e-05, 'epoch': 0.23}
{'loss': 1.4586, 'grad_norm': 7.0118303298950195, 'learning_rate': 4.621696025849067e-05, 'epoch': 0.23}
{'loss': 1.4719, 'grad_norm': 7.503939151763916, 'learning_rate': 4.621420895285858e-05, 'epoch': 0.23}
{'loss': 1.4621, 'grad_norm': 14.529010772705078, 'learning_rate': 4.621145764722649e-05, 'epoch': 0.23}
{'loss': 1.4885, 'grad_norm': 12.775609970092773, 'learning_rate': 4.62087063415944e-05, 'epoch': 0.23}
{'loss': 1.4628, 'grad_norm': 8.490446090698242, 'learning_rate': 4.6205955035962317e-05, 'epoch': 0.23}
{'loss': 1.4869, 'grad_norm': 10.967869758605957, 'learning_rate': 4.620320373033023e-05, 'epoch': 0.23}
{'loss': 1.4809, 'grad_norm': 8.537841796875, 'learning_rate': 4.620045242469814e-05, 'epoch': 0.23}
{'loss': 1.4762, 'grad_norm': 9.563363075256348, 'learning_rate': 4.6197701119066055e-05, 'epoch': 0.23}
{'loss': 1.4831, 'grad_norm': 13.707825660705566, 'learning_rate': 4.619494981343397e-05, 'epoch': 0.23}
{'loss': 1.4857, 'grad_norm': 8.526674270629883, 'learning_rate': 4.619219850780188e-05, 'epoch': 0.23}
{'loss': 1.5176, 'grad_norm': 6.382627487182617, 'learning_rate': 4.6189447202169794e-05, 'epoch': 0.23}
{'loss': 1.4633, 'grad_norm': 17.349328994750977, 'learning_rate': 4.61866958965377e-05, 'epoch': 0.23}
{'loss': 1.4895, 'grad_norm': 12.780815124511719, 'learning_rate': 4.618394459090562e-05, 'epoch': 0.23}
{'loss': 1.4759, 'grad_norm': 8.523032188415527, 'learning_rate': 4.6181193285273526e-05, 'epoch': 0.23}
{'loss': 1.4759, 'grad_norm': 10.701325416564941, 'learning_rate': 4.617844197964144e-05, 'epoch': 0.23}
{'loss': 1.4664, 'grad_norm': 8.213004112243652, 'learning_rate': 4.6175690674009356e-05, 'epoch': 0.23}
{'loss': 1.443, 'grad_norm': 10.548017501831055, 'learning_rate': 4.6172939368377264e-05, 'epoch': 0.23}
{'loss': 1.4859, 'grad_norm': 8.217314720153809, 'learning_rate': 4.617018806274518e-05, 'epoch': 0.23}
{'loss': 1.4543, 'grad_norm': 6.141848087310791, 'learning_rate': 4.6167436757113095e-05, 'epoch': 0.23}
{'loss': 1.4692, 'grad_norm': 8.64680290222168, 'learning_rate': 4.6164685451481e-05, 'epoch': 0.23}
{'loss': 1.4578, 'grad_norm': 10.007878303527832, 'learning_rate': 4.616193414584892e-05, 'epoch': 0.23}
{'loss': 1.4428, 'grad_norm': 9.764575004577637, 'learning_rate': 4.6159182840216827e-05, 'epoch': 0.23}
{'loss': 1.4595, 'grad_norm': 16.265975952148438, 'learning_rate': 4.6156431534584735e-05, 'epoch': 0.23}
{'loss': 1.4496, 'grad_norm': 9.682488441467285, 'learning_rate': 4.615368022895265e-05, 'epoch': 0.23}
{'loss': 1.4713, 'grad_norm': 8.668978691101074, 'learning_rate': 4.6150928923320565e-05, 'epoch': 0.23}
{'loss': 1.4626, 'grad_norm': 10.50040340423584, 'learning_rate': 4.6148177617688474e-05, 'epoch': 0.23}
{'loss': 1.4702, 'grad_norm': 11.752323150634766, 'learning_rate': 4.614542631205639e-05, 'epoch': 0.23}
{'loss': 1.4776, 'grad_norm': 9.522810935974121, 'learning_rate': 4.6142675006424304e-05, 'epoch': 0.23}
{'loss': 1.4657, 'grad_norm': 8.836688041687012, 'learning_rate': 4.613992370079221e-05, 'epoch': 0.23}
{'loss': 1.4878, 'grad_norm': 9.4932222366333, 'learning_rate': 4.613717239516013e-05, 'epoch': 0.23}
{'loss': 1.4851, 'grad_norm': 6.8865461349487305, 'learning_rate': 4.613442108952804e-05, 'epoch': 0.23}
{'loss': 1.4688, 'grad_norm': 8.226765632629395, 'learning_rate': 4.613166978389595e-05, 'epoch': 0.23}
{'loss': 1.4648, 'grad_norm': 6.919748783111572, 'learning_rate': 4.612891847826386e-05, 'epoch': 0.23}
{'loss': 1.4746, 'grad_norm': 8.012988090515137, 'learning_rate': 4.6126167172631774e-05, 'epoch': 0.23}
{'loss': 1.4957, 'grad_norm': 12.104554176330566, 'learning_rate': 4.612341586699969e-05, 'epoch': 0.23}
{'loss': 1.4494, 'grad_norm': 8.386191368103027, 'learning_rate': 4.61206645613676e-05, 'epoch': 0.23}
{'loss': 1.4939, 'grad_norm': 8.85997486114502, 'learning_rate': 4.611791325573551e-05, 'epoch': 0.23}
{'loss': 1.4568, 'grad_norm': 8.371213912963867, 'learning_rate': 4.611516195010342e-05, 'epoch': 0.23}
{'loss': 1.4757, 'grad_norm': 9.596835136413574, 'learning_rate': 4.611241064447134e-05, 'epoch': 0.23}
{'loss': 1.621, 'grad_norm': 8.08952808380127, 'learning_rate': 4.610965933883925e-05, 'epoch': 0.23}
{'loss': 1.481, 'grad_norm': 10.563883781433105, 'learning_rate': 4.610690803320716e-05, 'epoch': 0.23}
{'loss': 1.4854, 'grad_norm': 10.059640884399414, 'learning_rate': 4.610415672757507e-05, 'epoch': 0.23}
{'loss': 1.4711, 'grad_norm': 15.335091590881348, 'learning_rate': 4.6101405421942984e-05, 'epoch': 0.23}
{'loss': 1.4524, 'grad_norm': 11.274651527404785, 'learning_rate': 4.60986541163109e-05, 'epoch': 0.23}
{'loss': 1.5522, 'grad_norm': 8.781759262084961, 'learning_rate': 4.609590281067881e-05, 'epoch': 0.23}
{'loss': 1.5329, 'grad_norm': 9.327632904052734, 'learning_rate': 4.609315150504672e-05, 'epoch': 0.23}
{'loss': 1.4869, 'grad_norm': 10.452779769897461, 'learning_rate': 4.609040019941464e-05, 'epoch': 0.23}
{'loss': 1.4993, 'grad_norm': 10.846524238586426, 'learning_rate': 4.6087648893782546e-05, 'epoch': 0.23}
{'loss': 1.4536, 'grad_norm': 8.913724899291992, 'learning_rate': 4.608489758815046e-05, 'epoch': 0.23}
{'loss': 1.4802, 'grad_norm': 9.135912895202637, 'learning_rate': 4.6082146282518376e-05, 'epoch': 0.24}
{'loss': 1.4833, 'grad_norm': 7.868535041809082, 'learning_rate': 4.607939497688628e-05, 'epoch': 0.24}
{'loss': 1.4973, 'grad_norm': 9.592262268066406, 'learning_rate': 4.607664367125419e-05, 'epoch': 0.24}
{'loss': 1.4849, 'grad_norm': 7.722249507904053, 'learning_rate': 4.607389236562211e-05, 'epoch': 0.24}
{'loss': 1.4934, 'grad_norm': 8.138383865356445, 'learning_rate': 4.6071141059990016e-05, 'epoch': 0.24}
{'loss': 1.4633, 'grad_norm': 10.631941795349121, 'learning_rate': 4.606838975435793e-05, 'epoch': 0.24}
{'loss': 1.4582, 'grad_norm': 8.537137985229492, 'learning_rate': 4.606563844872585e-05, 'epoch': 0.24}
{'loss': 1.4979, 'grad_norm': 7.053224086761475, 'learning_rate': 4.6062887143093755e-05, 'epoch': 0.24}
{'loss': 1.5143, 'grad_norm': 9.510153770446777, 'learning_rate': 4.606013583746167e-05, 'epoch': 0.24}
{'loss': 1.498, 'grad_norm': 8.986166000366211, 'learning_rate': 4.6057384531829585e-05, 'epoch': 0.24}
{'loss': 1.4781, 'grad_norm': 8.281599998474121, 'learning_rate': 4.6054633226197494e-05, 'epoch': 0.24}
{'loss': 1.4963, 'grad_norm': 8.489436149597168, 'learning_rate': 4.60518819205654e-05, 'epoch': 0.24}
{'loss': 1.4633, 'grad_norm': 7.932962894439697, 'learning_rate': 4.604913061493332e-05, 'epoch': 0.24}
{'loss': 1.4889, 'grad_norm': 8.4878511428833, 'learning_rate': 4.604637930930123e-05, 'epoch': 0.24}
{'loss': 1.478, 'grad_norm': 7.318838596343994, 'learning_rate': 4.604362800366914e-05, 'epoch': 0.24}
{'loss': 1.4897, 'grad_norm': 7.214949607849121, 'learning_rate': 4.6040876698037056e-05, 'epoch': 0.24}
{'loss': 1.476, 'grad_norm': 5.823390007019043, 'learning_rate': 4.603812539240497e-05, 'epoch': 0.24}
{'loss': 1.4688, 'grad_norm': 8.012323379516602, 'learning_rate': 4.603537408677288e-05, 'epoch': 0.24}
{'loss': 1.4806, 'grad_norm': 8.988447189331055, 'learning_rate': 4.6032622781140795e-05, 'epoch': 0.24}
{'loss': 1.4723, 'grad_norm': 7.997750759124756, 'learning_rate': 4.602987147550871e-05, 'epoch': 0.24}
{'loss': 1.4804, 'grad_norm': 9.810667991638184, 'learning_rate': 4.602712016987662e-05, 'epoch': 0.24}
{'loss': 1.4895, 'grad_norm': 7.129015922546387, 'learning_rate': 4.6024368864244526e-05, 'epoch': 0.24}
{'loss': 1.4861, 'grad_norm': 9.675742149353027, 'learning_rate': 4.602161755861244e-05, 'epoch': 0.24}
{'loss': 1.4901, 'grad_norm': 7.425166606903076, 'learning_rate': 4.601886625298035e-05, 'epoch': 0.24}
{'loss': 1.4635, 'grad_norm': 7.185835361480713, 'learning_rate': 4.6016114947348265e-05, 'epoch': 0.24}
{'loss': 1.4748, 'grad_norm': 7.644229888916016, 'learning_rate': 4.601336364171618e-05, 'epoch': 0.24}
{'loss': 1.4366, 'grad_norm': 10.08251953125, 'learning_rate': 4.601061233608409e-05, 'epoch': 0.24}
{'loss': 1.452, 'grad_norm': 8.061382293701172, 'learning_rate': 4.6007861030452004e-05, 'epoch': 0.24}
{'loss': 1.4872, 'grad_norm': 7.733463764190674, 'learning_rate': 4.600510972481992e-05, 'epoch': 0.24}
{'loss': 1.4716, 'grad_norm': 43.97593307495117, 'learning_rate': 4.600235841918783e-05, 'epoch': 0.24}
{'loss': 1.4607, 'grad_norm': 10.977044105529785, 'learning_rate': 4.599960711355574e-05, 'epoch': 0.24}
{'loss': 1.4606, 'grad_norm': 7.237864017486572, 'learning_rate': 4.599685580792365e-05, 'epoch': 0.24}
{'loss': 1.475, 'grad_norm': 6.848369598388672, 'learning_rate': 4.5994104502291566e-05, 'epoch': 0.24}
{'loss': 1.4577, 'grad_norm': 9.732434272766113, 'learning_rate': 4.5991353196659474e-05, 'epoch': 0.24}
{'loss': 1.4608, 'grad_norm': 10.578377723693848, 'learning_rate': 4.598860189102739e-05, 'epoch': 0.24}
{'loss': 1.4854, 'grad_norm': 9.27191162109375, 'learning_rate': 4.5985850585395305e-05, 'epoch': 0.24}
{'loss': 1.4491, 'grad_norm': 6.6596221923828125, 'learning_rate': 4.598309927976321e-05, 'epoch': 0.24}
{'loss': 1.4692, 'grad_norm': 56.48979949951172, 'learning_rate': 4.598034797413113e-05, 'epoch': 0.24}
{'loss': 1.4868, 'grad_norm': 11.266121864318848, 'learning_rate': 4.597759666849904e-05, 'epoch': 0.24}
{'loss': 1.456, 'grad_norm': 8.077472686767578, 'learning_rate': 4.597484536286695e-05, 'epoch': 0.24}
{'loss': 1.4732, 'grad_norm': 4.380825519561768, 'learning_rate': 4.597209405723487e-05, 'epoch': 0.24}
{'loss': 1.4741, 'grad_norm': 11.372743606567383, 'learning_rate': 4.5969342751602775e-05, 'epoch': 0.24}
{'loss': 1.4568, 'grad_norm': 7.648576259613037, 'learning_rate': 4.5966591445970684e-05, 'epoch': 0.24}
{'loss': 1.4887, 'grad_norm': 7.626880168914795, 'learning_rate': 4.59638401403386e-05, 'epoch': 0.24}
{'loss': 1.5337, 'grad_norm': 8.329614639282227, 'learning_rate': 4.5961088834706514e-05, 'epoch': 0.24}
{'loss': 1.4842, 'grad_norm': 11.79711627960205, 'learning_rate': 4.595833752907442e-05, 'epoch': 0.24}
{'loss': 1.4787, 'grad_norm': 10.647479057312012, 'learning_rate': 4.595558622344234e-05, 'epoch': 0.24}
{'loss': 1.4864, 'grad_norm': 9.071717262268066, 'learning_rate': 4.595283491781025e-05, 'epoch': 0.24}
{'loss': 1.4706, 'grad_norm': 7.1928510665893555, 'learning_rate': 4.595008361217816e-05, 'epoch': 0.24}
{'loss': 1.4566, 'grad_norm': 9.749197959899902, 'learning_rate': 4.5947332306546076e-05, 'epoch': 0.24}
{'loss': 1.5413, 'grad_norm': 6.411102294921875, 'learning_rate': 4.5944581000913984e-05, 'epoch': 0.24}
{'loss': 1.4955, 'grad_norm': 9.609704971313477, 'learning_rate': 4.594182969528189e-05, 'epoch': 0.24}
{'loss': 1.4674, 'grad_norm': 8.786553382873535, 'learning_rate': 4.593907838964981e-05, 'epoch': 0.24}
{'loss': 1.4847, 'grad_norm': 8.995070457458496, 'learning_rate': 4.593632708401772e-05, 'epoch': 0.24}
{'loss': 1.4838, 'grad_norm': 12.607708930969238, 'learning_rate': 4.593357577838563e-05, 'epoch': 0.24}
{'loss': 1.4752, 'grad_norm': 23.049734115600586, 'learning_rate': 4.5930824472753547e-05, 'epoch': 0.24}
{'loss': 1.478, 'grad_norm': 8.028285026550293, 'learning_rate': 4.592807316712146e-05, 'epoch': 0.24}
{'loss': 1.4842, 'grad_norm': 6.952579975128174, 'learning_rate': 4.592532186148937e-05, 'epoch': 0.24}
{'loss': 1.4539, 'grad_norm': 8.546637535095215, 'learning_rate': 4.5922570555857285e-05, 'epoch': 0.24}
{'loss': 1.455, 'grad_norm': 9.245615005493164, 'learning_rate': 4.59198192502252e-05, 'epoch': 0.24}
{'loss': 1.4565, 'grad_norm': 9.009268760681152, 'learning_rate': 4.591706794459311e-05, 'epoch': 0.24}
{'loss': 1.4788, 'grad_norm': 10.611557960510254, 'learning_rate': 4.591431663896102e-05, 'epoch': 0.25}
{'loss': 1.4596, 'grad_norm': 8.485103607177734, 'learning_rate': 4.591156533332893e-05, 'epoch': 0.25}
{'loss': 1.4818, 'grad_norm': 6.814709186553955, 'learning_rate': 4.590881402769685e-05, 'epoch': 0.25}
{'loss': 1.479, 'grad_norm': 11.109850883483887, 'learning_rate': 4.5906062722064756e-05, 'epoch': 0.25}
{'loss': 1.4557, 'grad_norm': 9.295619010925293, 'learning_rate': 4.590331141643267e-05, 'epoch': 0.25}
{'loss': 1.4468, 'grad_norm': 9.908978462219238, 'learning_rate': 4.5900560110800586e-05, 'epoch': 0.25}
{'loss': 1.4931, 'grad_norm': 11.96489143371582, 'learning_rate': 4.5897808805168494e-05, 'epoch': 0.25}
{'loss': 1.4362, 'grad_norm': 8.942054748535156, 'learning_rate': 4.589505749953641e-05, 'epoch': 0.25}
{'loss': 1.48, 'grad_norm': 6.227571487426758, 'learning_rate': 4.5892306193904325e-05, 'epoch': 0.25}
{'loss': 1.4672, 'grad_norm': 7.482460975646973, 'learning_rate': 4.5889554888272226e-05, 'epoch': 0.25}
{'loss': 1.4719, 'grad_norm': 7.525750160217285, 'learning_rate': 4.588680358264014e-05, 'epoch': 0.25}
{'loss': 1.4723, 'grad_norm': 10.178977012634277, 'learning_rate': 4.5884052277008057e-05, 'epoch': 0.25}
{'loss': 1.4592, 'grad_norm': 10.663661003112793, 'learning_rate': 4.5881300971375965e-05, 'epoch': 0.25}
{'loss': 1.4676, 'grad_norm': 7.286592960357666, 'learning_rate': 4.587854966574388e-05, 'epoch': 0.25}
{'loss': 1.4719, 'grad_norm': 33.26529312133789, 'learning_rate': 4.5875798360111795e-05, 'epoch': 0.25}
{'loss': 1.4477, 'grad_norm': 8.513075828552246, 'learning_rate': 4.5873047054479704e-05, 'epoch': 0.25}
{'loss': 1.4535, 'grad_norm': 8.451760292053223, 'learning_rate': 4.587029574884762e-05, 'epoch': 0.25}
{'loss': 1.51, 'grad_norm': 8.794035911560059, 'learning_rate': 4.5867544443215534e-05, 'epoch': 0.25}
{'loss': 1.4879, 'grad_norm': 9.401087760925293, 'learning_rate': 4.586479313758344e-05, 'epoch': 0.25}
{'loss': 1.471, 'grad_norm': 9.223701477050781, 'learning_rate': 4.586204183195135e-05, 'epoch': 0.25}
{'loss': 1.5255, 'grad_norm': 5.832275390625, 'learning_rate': 4.5859290526319266e-05, 'epoch': 0.25}
{'loss': 1.4951, 'grad_norm': 9.344109535217285, 'learning_rate': 4.585653922068718e-05, 'epoch': 0.25}
{'loss': 1.4662, 'grad_norm': 9.746687889099121, 'learning_rate': 4.585378791505509e-05, 'epoch': 0.25}
{'loss': 1.4579, 'grad_norm': 8.846099853515625, 'learning_rate': 4.5851036609423004e-05, 'epoch': 0.25}
{'loss': 1.4829, 'grad_norm': 6.005495548248291, 'learning_rate': 4.584828530379092e-05, 'epoch': 0.25}
{'loss': 1.4746, 'grad_norm': 7.332201957702637, 'learning_rate': 4.584553399815883e-05, 'epoch': 0.25}
{'loss': 1.4881, 'grad_norm': 14.217522621154785, 'learning_rate': 4.584278269252674e-05, 'epoch': 0.25}
{'loss': 1.4796, 'grad_norm': 9.945682525634766, 'learning_rate': 4.584003138689466e-05, 'epoch': 0.25}
{'loss': 1.4407, 'grad_norm': 7.896058559417725, 'learning_rate': 4.5837280081262567e-05, 'epoch': 0.25}
{'loss': 1.4931, 'grad_norm': 10.546810150146484, 'learning_rate': 4.5834528775630475e-05, 'epoch': 0.25}
{'loss': 1.4621, 'grad_norm': 8.565665245056152, 'learning_rate': 4.583177746999839e-05, 'epoch': 0.25}
{'loss': 1.4716, 'grad_norm': 8.441402435302734, 'learning_rate': 4.58290261643663e-05, 'epoch': 0.25}
{'loss': 1.5257, 'grad_norm': 9.486259460449219, 'learning_rate': 4.5826274858734214e-05, 'epoch': 0.25}
{'loss': 1.4517, 'grad_norm': 11.916622161865234, 'learning_rate': 4.582352355310213e-05, 'epoch': 0.25}
{'loss': 1.4846, 'grad_norm': 13.474347114562988, 'learning_rate': 4.582077224747004e-05, 'epoch': 0.25}
{'loss': 1.4706, 'grad_norm': 17.845237731933594, 'learning_rate': 4.581802094183795e-05, 'epoch': 0.25}
{'loss': 1.4573, 'grad_norm': 11.083505630493164, 'learning_rate': 4.581526963620587e-05, 'epoch': 0.25}
{'loss': 1.4845, 'grad_norm': 15.855786323547363, 'learning_rate': 4.5812518330573776e-05, 'epoch': 0.25}
{'loss': 1.5011, 'grad_norm': 6.20281982421875, 'learning_rate': 4.580976702494169e-05, 'epoch': 0.25}
{'loss': 1.4584, 'grad_norm': 9.439833641052246, 'learning_rate': 4.58070157193096e-05, 'epoch': 0.25}
{'loss': 1.46, 'grad_norm': 5.314873695373535, 'learning_rate': 4.5804264413677514e-05, 'epoch': 0.25}
{'loss': 1.4867, 'grad_norm': 9.55517864227295, 'learning_rate': 4.580151310804542e-05, 'epoch': 0.25}
{'loss': 1.4698, 'grad_norm': 10.989895820617676, 'learning_rate': 4.579876180241334e-05, 'epoch': 0.25}
{'loss': 1.4513, 'grad_norm': 10.390304565429688, 'learning_rate': 4.579601049678125e-05, 'epoch': 0.25}
{'loss': 1.4799, 'grad_norm': 10.698821067810059, 'learning_rate': 4.579325919114916e-05, 'epoch': 0.25}
{'loss': 1.4894, 'grad_norm': 8.512308120727539, 'learning_rate': 4.579050788551708e-05, 'epoch': 0.25}
{'loss': 1.4593, 'grad_norm': 8.42155647277832, 'learning_rate': 4.5787756579884985e-05, 'epoch': 0.25}
{'loss': 1.4615, 'grad_norm': 14.282062530517578, 'learning_rate': 4.57850052742529e-05, 'epoch': 0.25}
{'loss': 1.5088, 'grad_norm': 7.208677291870117, 'learning_rate': 4.578225396862081e-05, 'epoch': 0.25}
{'loss': 1.485, 'grad_norm': 8.498762130737305, 'learning_rate': 4.5779502662988724e-05, 'epoch': 0.25}
{'loss': 1.4836, 'grad_norm': 7.475222110748291, 'learning_rate': 4.577675135735663e-05, 'epoch': 0.25}
{'loss': 1.458, 'grad_norm': 9.506437301635742, 'learning_rate': 4.577400005172455e-05, 'epoch': 0.25}
{'loss': 1.4508, 'grad_norm': 8.127285957336426, 'learning_rate': 4.577124874609246e-05, 'epoch': 0.25}
{'loss': 1.458, 'grad_norm': 8.466217994689941, 'learning_rate': 4.576849744046037e-05, 'epoch': 0.25}
{'loss': 1.4654, 'grad_norm': 6.736040115356445, 'learning_rate': 4.5765746134828286e-05, 'epoch': 0.25}
{'loss': 1.4536, 'grad_norm': 6.578516006469727, 'learning_rate': 4.57629948291962e-05, 'epoch': 0.25}
{'loss': 1.4929, 'grad_norm': 14.460539817810059, 'learning_rate': 4.576024352356411e-05, 'epoch': 0.25}
{'loss': 1.4552, 'grad_norm': 8.351968765258789, 'learning_rate': 4.5757492217932025e-05, 'epoch': 0.25}
{'loss': 1.4671, 'grad_norm': 9.3458833694458, 'learning_rate': 4.575474091229993e-05, 'epoch': 0.25}
{'loss': 1.4532, 'grad_norm': 8.195968627929688, 'learning_rate': 4.575198960666784e-05, 'epoch': 0.25}
{'loss': 1.4591, 'grad_norm': 8.927186012268066, 'learning_rate': 4.5749238301035756e-05, 'epoch': 0.26}
{'loss': 1.4774, 'grad_norm': 10.910115242004395, 'learning_rate': 4.574648699540367e-05, 'epoch': 0.26}
{'loss': 1.4785, 'grad_norm': 7.072763919830322, 'learning_rate': 4.574373568977158e-05, 'epoch': 0.26}
{'loss': 1.4569, 'grad_norm': 7.366036891937256, 'learning_rate': 4.5740984384139495e-05, 'epoch': 0.26}
{'loss': 1.49, 'grad_norm': 8.703786849975586, 'learning_rate': 4.573823307850741e-05, 'epoch': 0.26}
{'loss': 1.4391, 'grad_norm': 9.07820987701416, 'learning_rate': 4.573548177287532e-05, 'epoch': 0.26}
{'loss': 1.4828, 'grad_norm': 8.27930736541748, 'learning_rate': 4.5732730467243234e-05, 'epoch': 0.26}
{'loss': 1.4595, 'grad_norm': 8.852511405944824, 'learning_rate': 4.572997916161115e-05, 'epoch': 0.26}
{'loss': 1.4689, 'grad_norm': 12.488775253295898, 'learning_rate': 4.572722785597906e-05, 'epoch': 0.26}
{'loss': 1.4607, 'grad_norm': 6.629610538482666, 'learning_rate': 4.5724476550346966e-05, 'epoch': 0.26}
{'loss': 1.4758, 'grad_norm': 8.492959022521973, 'learning_rate': 4.572172524471488e-05, 'epoch': 0.26}
{'loss': 1.471, 'grad_norm': 7.4165873527526855, 'learning_rate': 4.5718973939082796e-05, 'epoch': 0.26}
{'loss': 1.4748, 'grad_norm': 7.831920623779297, 'learning_rate': 4.5716222633450704e-05, 'epoch': 0.26}
{'loss': 1.4591, 'grad_norm': 9.18654727935791, 'learning_rate': 4.571347132781862e-05, 'epoch': 0.26}
{'loss': 1.4688, 'grad_norm': 8.44622802734375, 'learning_rate': 4.5710720022186535e-05, 'epoch': 0.26}
{'loss': 1.4654, 'grad_norm': 13.243745803833008, 'learning_rate': 4.570796871655444e-05, 'epoch': 0.26}
{'loss': 1.4564, 'grad_norm': 7.853121280670166, 'learning_rate': 4.570521741092236e-05, 'epoch': 0.26}
{'loss': 1.485, 'grad_norm': 17.77979850769043, 'learning_rate': 4.570246610529027e-05, 'epoch': 0.26}
{'loss': 1.458, 'grad_norm': 8.105718612670898, 'learning_rate': 4.5699714799658175e-05, 'epoch': 0.26}
{'loss': 1.4546, 'grad_norm': 7.9392991065979, 'learning_rate': 4.569696349402609e-05, 'epoch': 0.26}
{'loss': 1.4706, 'grad_norm': 7.827721118927002, 'learning_rate': 4.5694212188394005e-05, 'epoch': 0.26}
{'loss': 1.4553, 'grad_norm': 7.341506004333496, 'learning_rate': 4.5691460882761913e-05, 'epoch': 0.26}
{'loss': 1.483, 'grad_norm': 5.726408004760742, 'learning_rate': 4.568870957712983e-05, 'epoch': 0.26}
{'loss': 1.4786, 'grad_norm': 7.557697296142578, 'learning_rate': 4.5685958271497744e-05, 'epoch': 0.26}
{'loss': 1.4664, 'grad_norm': 9.84631633758545, 'learning_rate': 4.568320696586565e-05, 'epoch': 0.26}
{'loss': 1.4561, 'grad_norm': 7.206833362579346, 'learning_rate': 4.568045566023357e-05, 'epoch': 0.26}
{'loss': 1.4667, 'grad_norm': 7.3498759269714355, 'learning_rate': 4.567770435460148e-05, 'epoch': 0.26}
{'loss': 1.4372, 'grad_norm': 7.204351902008057, 'learning_rate': 4.567495304896939e-05, 'epoch': 0.26}
{'loss': 1.4579, 'grad_norm': 8.011987686157227, 'learning_rate': 4.56722017433373e-05, 'epoch': 0.26}
{'loss': 1.474, 'grad_norm': 12.96658706665039, 'learning_rate': 4.5669450437705214e-05, 'epoch': 0.26}
{'loss': 1.4743, 'grad_norm': 8.299479484558105, 'learning_rate': 4.566669913207313e-05, 'epoch': 0.26}
{'loss': 1.4832, 'grad_norm': 9.075163841247559, 'learning_rate': 4.566394782644104e-05, 'epoch': 0.26}
{'loss': 1.4414, 'grad_norm': 7.752557277679443, 'learning_rate': 4.566119652080895e-05, 'epoch': 0.26}
{'loss': 1.4849, 'grad_norm': 7.062743186950684, 'learning_rate': 4.565844521517687e-05, 'epoch': 0.26}
{'loss': 1.4287, 'grad_norm': 7.927086353302002, 'learning_rate': 4.5655693909544776e-05, 'epoch': 0.26}
{'loss': 1.438, 'grad_norm': 9.065506935119629, 'learning_rate': 4.565294260391269e-05, 'epoch': 0.26}
{'loss': 1.4375, 'grad_norm': 9.277286529541016, 'learning_rate': 4.565019129828061e-05, 'epoch': 0.26}
{'loss': 1.4709, 'grad_norm': 12.99593448638916, 'learning_rate': 4.564743999264851e-05, 'epoch': 0.26}
{'loss': 1.4572, 'grad_norm': 8.548603057861328, 'learning_rate': 4.5644688687016424e-05, 'epoch': 0.26}
{'loss': 1.4526, 'grad_norm': 7.3713459968566895, 'learning_rate': 4.564193738138434e-05, 'epoch': 0.26}
{'loss': 1.4613, 'grad_norm': 9.396247863769531, 'learning_rate': 4.563918607575225e-05, 'epoch': 0.26}
{'loss': 1.473, 'grad_norm': 8.459238052368164, 'learning_rate': 4.563643477012016e-05, 'epoch': 0.26}
{'loss': 1.4688, 'grad_norm': 12.211153984069824, 'learning_rate': 4.563368346448808e-05, 'epoch': 0.26}
{'loss': 1.4439, 'grad_norm': 9.902155876159668, 'learning_rate': 4.5630932158855986e-05, 'epoch': 0.26}
{'loss': 1.4753, 'grad_norm': 10.116180419921875, 'learning_rate': 4.56281808532239e-05, 'epoch': 0.26}
{'loss': 1.4399, 'grad_norm': 6.7182745933532715, 'learning_rate': 4.5625429547591816e-05, 'epoch': 0.26}
{'loss': 1.4645, 'grad_norm': 5.847872257232666, 'learning_rate': 4.5622678241959724e-05, 'epoch': 0.26}
{'loss': 1.4614, 'grad_norm': 10.051958084106445, 'learning_rate': 4.561992693632763e-05, 'epoch': 0.26}
{'loss': 1.469, 'grad_norm': 10.327495574951172, 'learning_rate': 4.561717563069555e-05, 'epoch': 0.26}
{'loss': 1.4479, 'grad_norm': 8.083680152893066, 'learning_rate': 4.5614424325063456e-05, 'epoch': 0.26}
{'loss': 1.4538, 'grad_norm': 9.374457359313965, 'learning_rate': 4.561167301943137e-05, 'epoch': 0.26}
{'loss': 1.4661, 'grad_norm': 6.288130760192871, 'learning_rate': 4.5608921713799287e-05, 'epoch': 0.26}
{'loss': 1.4756, 'grad_norm': 9.694746017456055, 'learning_rate': 4.5606170408167195e-05, 'epoch': 0.26}
{'loss': 1.4464, 'grad_norm': 4.56768274307251, 'learning_rate': 4.560341910253511e-05, 'epoch': 0.26}
{'loss': 1.4619, 'grad_norm': 10.027815818786621, 'learning_rate': 4.5600667796903025e-05, 'epoch': 0.26}
{'loss': 1.4628, 'grad_norm': 8.816747665405273, 'learning_rate': 4.5597916491270934e-05, 'epoch': 0.26}
{'loss': 1.4552, 'grad_norm': 7.834435939788818, 'learning_rate': 4.559516518563885e-05, 'epoch': 0.26}
{'loss': 1.4474, 'grad_norm': 7.945765972137451, 'learning_rate': 4.559241388000676e-05, 'epoch': 0.26}
{'loss': 1.4872, 'grad_norm': 6.999059677124023, 'learning_rate': 4.558966257437467e-05, 'epoch': 0.26}
{'loss': 1.4721, 'grad_norm': 9.312894821166992, 'learning_rate': 4.558691126874258e-05, 'epoch': 0.26}
{'loss': 1.457, 'grad_norm': 8.400811195373535, 'learning_rate': 4.5584159963110496e-05, 'epoch': 0.26}
{'loss': 1.465, 'grad_norm': 10.949821472167969, 'learning_rate': 4.558140865747841e-05, 'epoch': 0.27}
{'loss': 1.4401, 'grad_norm': 8.06319808959961, 'learning_rate': 4.557865735184632e-05, 'epoch': 0.27}
{'loss': 1.4654, 'grad_norm': 8.923135757446289, 'learning_rate': 4.5575906046214234e-05, 'epoch': 0.27}
{'loss': 1.4549, 'grad_norm': 9.015380859375, 'learning_rate': 4.557315474058215e-05, 'epoch': 0.27}
{'loss': 1.4787, 'grad_norm': 9.590959548950195, 'learning_rate': 4.557040343495006e-05, 'epoch': 0.27}
{'loss': 1.4531, 'grad_norm': 8.33353042602539, 'learning_rate': 4.556765212931797e-05, 'epoch': 0.27}
{'loss': 1.4724, 'grad_norm': 7.695148468017578, 'learning_rate': 4.556490082368588e-05, 'epoch': 0.27}
{'loss': 1.4716, 'grad_norm': 8.761918067932129, 'learning_rate': 4.556214951805379e-05, 'epoch': 0.27}
{'loss': 1.4627, 'grad_norm': 7.387304306030273, 'learning_rate': 4.5559398212421705e-05, 'epoch': 0.27}
{'loss': 1.4631, 'grad_norm': 8.911149024963379, 'learning_rate': 4.555664690678962e-05, 'epoch': 0.27}
{'loss': 1.4607, 'grad_norm': 8.053589820861816, 'learning_rate': 4.555389560115753e-05, 'epoch': 0.27}
{'loss': 1.4609, 'grad_norm': 8.217938423156738, 'learning_rate': 4.5551144295525444e-05, 'epoch': 0.27}
{'loss': 1.4511, 'grad_norm': 8.838197708129883, 'learning_rate': 4.554839298989336e-05, 'epoch': 0.27}
{'loss': 1.4894, 'grad_norm': 7.156284809112549, 'learning_rate': 4.554564168426127e-05, 'epoch': 0.27}
{'loss': 1.4672, 'grad_norm': 8.701504707336426, 'learning_rate': 4.554289037862918e-05, 'epoch': 0.27}
{'loss': 1.462, 'grad_norm': 9.391708374023438, 'learning_rate': 4.55401390729971e-05, 'epoch': 0.27}
{'loss': 1.4544, 'grad_norm': 8.357171058654785, 'learning_rate': 4.5537387767365006e-05, 'epoch': 0.27}
{'loss': 1.4463, 'grad_norm': 10.056334495544434, 'learning_rate': 4.5534636461732914e-05, 'epoch': 0.27}
{'loss': 1.4713, 'grad_norm': 10.274479866027832, 'learning_rate': 4.553188515610083e-05, 'epoch': 0.27}
{'loss': 1.4389, 'grad_norm': 9.474105834960938, 'learning_rate': 4.5529133850468744e-05, 'epoch': 0.27}
{'loss': 1.4442, 'grad_norm': 6.790122032165527, 'learning_rate': 4.552638254483665e-05, 'epoch': 0.27}
{'loss': 1.4323, 'grad_norm': 8.092194557189941, 'learning_rate': 4.552363123920457e-05, 'epoch': 0.27}
{'loss': 1.4805, 'grad_norm': 7.500400066375732, 'learning_rate': 4.552087993357248e-05, 'epoch': 0.27}
{'loss': 1.4368, 'grad_norm': 8.937694549560547, 'learning_rate': 4.551812862794039e-05, 'epoch': 0.27}
{'loss': 1.4585, 'grad_norm': 10.50197696685791, 'learning_rate': 4.5515377322308307e-05, 'epoch': 0.27}
{'loss': 1.4597, 'grad_norm': 6.507939338684082, 'learning_rate': 4.551262601667622e-05, 'epoch': 0.27}
{'loss': 1.4812, 'grad_norm': 8.089361190795898, 'learning_rate': 4.550987471104412e-05, 'epoch': 0.27}
{'loss': 1.4727, 'grad_norm': 11.621830940246582, 'learning_rate': 4.550712340541204e-05, 'epoch': 0.27}
{'loss': 1.468, 'grad_norm': 7.5070390701293945, 'learning_rate': 4.5504372099779954e-05, 'epoch': 0.27}
{'loss': 1.4455, 'grad_norm': 7.025522708892822, 'learning_rate': 4.550162079414786e-05, 'epoch': 0.27}
{'loss': 1.4483, 'grad_norm': 8.620267868041992, 'learning_rate': 4.549886948851578e-05, 'epoch': 0.27}
{'loss': 1.4404, 'grad_norm': 10.082969665527344, 'learning_rate': 4.549611818288369e-05, 'epoch': 0.27}
{'loss': 1.4558, 'grad_norm': 9.269762992858887, 'learning_rate': 4.54933668772516e-05, 'epoch': 0.27}
{'loss': 1.4418, 'grad_norm': 8.844417572021484, 'learning_rate': 4.5490615571619516e-05, 'epoch': 0.27}
{'loss': 1.4794, 'grad_norm': 8.270543098449707, 'learning_rate': 4.548786426598743e-05, 'epoch': 0.27}
{'loss': 1.431, 'grad_norm': 8.304194450378418, 'learning_rate': 4.548511296035534e-05, 'epoch': 0.27}
{'loss': 1.4571, 'grad_norm': 9.044967651367188, 'learning_rate': 4.548236165472325e-05, 'epoch': 0.27}
{'loss': 1.4623, 'grad_norm': 10.538127899169922, 'learning_rate': 4.547961034909116e-05, 'epoch': 0.27}
{'loss': 1.444, 'grad_norm': 7.597245216369629, 'learning_rate': 4.547685904345908e-05, 'epoch': 0.27}
{'loss': 1.4376, 'grad_norm': 7.364052772521973, 'learning_rate': 4.5474107737826986e-05, 'epoch': 0.27}
{'loss': 1.437, 'grad_norm': 6.774776458740234, 'learning_rate': 4.54713564321949e-05, 'epoch': 0.27}
{'loss': 1.4736, 'grad_norm': 10.122713088989258, 'learning_rate': 4.546860512656281e-05, 'epoch': 0.27}
{'loss': 1.4498, 'grad_norm': 8.690244674682617, 'learning_rate': 4.5465853820930725e-05, 'epoch': 0.27}
{'loss': 1.4394, 'grad_norm': 10.221277236938477, 'learning_rate': 4.546310251529864e-05, 'epoch': 0.27}
{'loss': 1.4542, 'grad_norm': 13.029943466186523, 'learning_rate': 4.546035120966655e-05, 'epoch': 0.27}
{'loss': 1.4494, 'grad_norm': 9.339994430541992, 'learning_rate': 4.545759990403446e-05, 'epoch': 0.27}
{'loss': 1.4382, 'grad_norm': 7.779630661010742, 'learning_rate': 4.545484859840237e-05, 'epoch': 0.27}
{'loss': 1.4629, 'grad_norm': 7.528447151184082, 'learning_rate': 4.545209729277029e-05, 'epoch': 0.27}
{'loss': 1.4543, 'grad_norm': 7.721797466278076, 'learning_rate': 4.5449345987138196e-05, 'epoch': 0.27}
{'loss': 1.4531, 'grad_norm': 10.655647277832031, 'learning_rate': 4.544659468150611e-05, 'epoch': 0.27}
{'loss': 1.4627, 'grad_norm': 8.45132827758789, 'learning_rate': 4.5443843375874026e-05, 'epoch': 0.27}
{'loss': 1.4772, 'grad_norm': 10.250021934509277, 'learning_rate': 4.5441092070241934e-05, 'epoch': 0.27}
{'loss': 1.4668, 'grad_norm': 11.772461891174316, 'learning_rate': 4.543834076460985e-05, 'epoch': 0.27}
{'loss': 1.4543, 'grad_norm': 6.897887706756592, 'learning_rate': 4.5435589458977765e-05, 'epoch': 0.27}
{'loss': 1.4543, 'grad_norm': 8.563858032226562, 'learning_rate': 4.543283815334567e-05, 'epoch': 0.27}
{'loss': 1.4415, 'grad_norm': 9.569244384765625, 'learning_rate': 4.543008684771358e-05, 'epoch': 0.27}
{'loss': 1.4538, 'grad_norm': 7.998226642608643, 'learning_rate': 4.5427335542081496e-05, 'epoch': 0.27}
{'loss': 1.4512, 'grad_norm': 11.40449047088623, 'learning_rate': 4.5424584236449405e-05, 'epoch': 0.27}
{'loss': 1.4382, 'grad_norm': 7.025571346282959, 'learning_rate': 4.542183293081732e-05, 'epoch': 0.27}
{'loss': 1.4536, 'grad_norm': 6.469700336456299, 'learning_rate': 4.5419081625185235e-05, 'epoch': 0.27}
{'loss': 1.4515, 'grad_norm': 7.555152416229248, 'learning_rate': 4.5416330319553143e-05, 'epoch': 0.28}
{'loss': 1.4479, 'grad_norm': 7.006507396697998, 'learning_rate': 4.541357901392106e-05, 'epoch': 0.28}
{'loss': 1.4446, 'grad_norm': 8.472725868225098, 'learning_rate': 4.5410827708288974e-05, 'epoch': 0.28}
{'loss': 1.4687, 'grad_norm': 8.722235679626465, 'learning_rate': 4.540807640265688e-05, 'epoch': 0.28}
{'loss': 1.4664, 'grad_norm': 9.367838859558105, 'learning_rate': 4.54053250970248e-05, 'epoch': 0.28}
{'loss': 1.4478, 'grad_norm': 8.149343490600586, 'learning_rate': 4.5402573791392706e-05, 'epoch': 0.28}
{'loss': 1.4469, 'grad_norm': 7.9044904708862305, 'learning_rate': 4.539982248576062e-05, 'epoch': 0.28}
{'loss': 1.4693, 'grad_norm': 9.665281295776367, 'learning_rate': 4.539707118012853e-05, 'epoch': 0.28}
{'loss': 1.4385, 'grad_norm': 7.060789585113525, 'learning_rate': 4.5394319874496444e-05, 'epoch': 0.28}
{'loss': 1.4458, 'grad_norm': 7.036242961883545, 'learning_rate': 4.539156856886436e-05, 'epoch': 0.28}
{'loss': 1.436, 'grad_norm': 8.203239440917969, 'learning_rate': 4.538881726323227e-05, 'epoch': 0.28}
{'loss': 1.4614, 'grad_norm': 8.283976554870605, 'learning_rate': 4.538606595760018e-05, 'epoch': 0.28}
{'loss': 1.4668, 'grad_norm': 7.3492350578308105, 'learning_rate': 4.53833146519681e-05, 'epoch': 0.28}
{'loss': 1.444, 'grad_norm': 10.535237312316895, 'learning_rate': 4.5380563346336006e-05, 'epoch': 0.28}
{'loss': 1.4467, 'grad_norm': 10.418097496032715, 'learning_rate': 4.537781204070392e-05, 'epoch': 0.28}
{'loss': 1.4527, 'grad_norm': 10.546502113342285, 'learning_rate': 4.537506073507183e-05, 'epoch': 0.28}
{'loss': 1.4491, 'grad_norm': 8.340408325195312, 'learning_rate': 4.537230942943974e-05, 'epoch': 0.28}
{'loss': 1.4521, 'grad_norm': 6.847714900970459, 'learning_rate': 4.5369558123807653e-05, 'epoch': 0.28}
{'loss': 1.4653, 'grad_norm': 7.7623748779296875, 'learning_rate': 4.536680681817557e-05, 'epoch': 0.28}
{'loss': 1.4409, 'grad_norm': 8.91409683227539, 'learning_rate': 4.536405551254348e-05, 'epoch': 0.28}
{'loss': 1.4733, 'grad_norm': 7.391883373260498, 'learning_rate': 4.536130420691139e-05, 'epoch': 0.28}
{'loss': 1.4597, 'grad_norm': 10.386221885681152, 'learning_rate': 4.535855290127931e-05, 'epoch': 0.28}
{'loss': 1.4463, 'grad_norm': 6.121020793914795, 'learning_rate': 4.5355801595647216e-05, 'epoch': 0.28}
{'loss': 1.4243, 'grad_norm': 10.046622276306152, 'learning_rate': 4.535305029001513e-05, 'epoch': 0.28}
{'loss': 1.4418, 'grad_norm': 11.562287330627441, 'learning_rate': 4.5350298984383046e-05, 'epoch': 0.28}

}

import re
import pandas as pd
import matplotlib.pyplot as plt

# If your cell `data = {...}` exists and contains strings like "{'loss': 1.45, ...}"
losses = []
epochs = []
for i, item in enumerate(data):
    if not isinstance(item, str):
        continue
    m_loss = re.search(r"'loss':\s*([+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?|inf)", item)
    m_epoch = re.search(r"'epoch':\s*([0-9]+(?:\.[0-9]+)?)", item)
    if not m_loss:
        continue
    val = m_loss.group(1)
    loss = float('inf') if val == 'inf' else float(val)
    losses.append(loss)
    epochs.append(float(m_epoch.group(1)) if m_epoch else float(i))

# Build DataFrame, sort by epoch
df = pd.DataFrame({'epoch': epochs, 'loss': losses})
df = df.sort_values('epoch').reset_index(drop=True)

# Plot raw loss and smoothed loss (rolling window)
plt.figure(figsize=(10,4))
plt.plot(df.index, df['loss'], marker='.', ms=3, lw=0.6, alpha=0.8, label='loss')
# replace infinities for smoothing, forward-fill then rolling mean
series_for_smooth = df['loss'].replace([float('inf')], pd.NA).fillna(method='ffill').astype(float)
df['loss_smooth'] = series_for_smooth.rolling(window=50, min_periods=1).mean()
plt.plot(df.index, df['loss_smooth'], color='orange', lw=1.2, label='rolling(50)')
plt.xlabel('Iteration (index)')
plt.ylabel('Loss')
plt.title('Training loss over iterations')
plt.grid(alpha=0.3)
plt.legend()
plt.show()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1725159904.py, line 3)